In [ ]:
# Parameters
SCENARIO_IDS = ["02", "03", "06", "10"]
RESOLUTION = "3H"
YEARS = ["2030", "2035"]

# Scenario Analysis - "Grid modelling to assess electrofuels supply potential - The impact of electrofuels on the US electricity grid"

This notebook reports the results of preliminary runs for the scenarios defined in the table [here](https://docs.google.com/document/d/1ssc5ilxEhEYYjFDCo5cIAgP7zSRcO4uVUXjxbyfR88Q/edit?tab=t.0). In this notebook, a single scenario is analyzed. Another notebook is available for multi-scenario comparison.


| | | **1** | **2** | **3** | **4** | **5** | **6** | **7** | **8** | **9** | **10** |
|---|---|---|---|---|---|---|---|---|---|---|---|
|  |  | **No e-kerosene mandate** | **ReFuel EU** | **ReFuel EU+** | **ReFuel EU-** | **High climate ambition & No e-kerosene mandate** | **High climate ambition &ReFuel EU** | **Optimistic electricity generation costs** | **Optimistic electrolyzer costs** | **Conservative electrolyzer costs** | **Biogenic point-source CO2 only** |
| **Hydrogen policy** | **Application of 45V pillars** | Yes* | Yes | Yes | Yes | Yes* | Yes | Yes | Yes | Yes | No |
| **Aviation sector** | **Demand** | Central | Central | Central | Central | Central | Central | Central | Central | Central | Central |
|  | **e-kerosene mandate** | No | ReFuel EU | ReFuel EU + | ReFuel EU - | No | ReFuel EU | ReFuel EU | ReFuel EU | ReFuel EU | ReFuel EU |
| **Demand projections** | **Electrification** | Medium | Medium | Medium | Medium | High | High | Medium | Medium | Medium | Medium |
|  | **Sectoral demand** | Reference | Reference | Reference | Reference | High Economic Growth | High Economic Growth | Reference | Reference | Reference | Reference |
| **Technology costs** | **Electricity generation and storage** | Moderate + tax credits | Moderate + tax credits | Moderate + tax credits | Moderate + tax credits | Moderate + tax credits (IRA 2022) | Moderate + tax credits (IRA 2022) | Advanced + tax credits | Moderate + tax credits | Moderate + tax credits | Moderate + tax credits |
|  | **Electrolysis** | Medium | Medium | Medium | Medium | Medium + tax credits (IRA 2022) | Medium + tax credits (IRA 2022) | Medium | Low | High | Medium |
|  | **DAC** | Medium + tax credits | Medium + tax credits | Medium + tax credits | Medium + tax credits | Medium + tax credits (IRA 2022) | Medium + tax credits (IRA 2022) | Medium + tax credits | Medium + tax credits | Medium + tax credits | Medium + tax credits |
|  | **Point-source CO2 capture** | High + tax credits | High + tax credits | High + tax credits | High + tax credits | High + tax credits (IRA 2022) | High + tax credits (IRA 2022) | High + tax credits | High + tax credits | High + tax credits | High + tax credits |
| **CO2 supply constraint** | **CO2 supply** | All point sources & DAC | All point sources & DAC | All point sources & DAC | All point sources & DAC | Biogenic point sources & DAC | Biogenic point sources & DAC | All point sources & DAC | All point sources & DAC | All point sources & DAC | Biogenic point sources & DAC |
| **Power sector development** | **Transmission capacity expansion** | No new expansion | No new expansion | No new expansion | No new expansion | Cost-optimal | Cost-optimal | No new expansion | No new expansion | No new expansion | No new expansion |
|  | **Policies for electricity generation** | Current State policies | Current State policies | Current State policies | Current State policies | Current State policies + 90% clean electricity by 2040 | Current State policies + 90% clean electricity by 2040 | Current State policies | Current State policies | Current State policies | Current State policies |

\* No demand for e-kerosene actually means no need for 45V pillars


---

## 1. Setup and Data Loading

*This section handles the initial setup, including importing necessary libraries and loading the solved PyPSA network data for each scenario.*

### 1.1. Import Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pycountry
from collections import defaultdict

import pypsa
from pypsa.plot import add_legend_circles, add_legend_lines, add_legend_patches
from pathlib import Path

import cartopy.crs as ccrs  # For plotting maps
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import box
from matplotlib.offsetbox import AnnotationBbox, AuxTransformBox
from matplotlib.patches import Wedge
import matplotlib.path as mpath
import matplotlib.transforms as mtransforms
import matplotlib.lines as mlines
from matplotlib.legend_handler import HandlerPatch
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from IPython.display import display, HTML, Markdown
from functools import reduce

import plotly.express as px
import plotly.graph_objects as go

from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
from datetime import datetime
import getpass, sys, platform

from shapely.geometry import LineString
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import yaml


from _helpers import *

In [ ]:
with open("plotting.yaml", "r") as file:
    plotting = yaml.safe_load(file)

tech_colors = plotting["tech_colors"]
nice_names = plotting["nice_names"]
rename_techs_capex = plotting["rename_tech_capex"]
rename_techs_opex = plotting["rename_tech_opex"]
rename_tech_colors = plotting["renamed_tech_colors"]
categories_capex = plotting["categories_capex"]
categories_opex = plotting["categories_opex"]
tech_power_color = plotting["tech_power_color"]
nice_names_power = plotting["nice_names_power"]
tech_power_order = plotting["tech_power_order"]
emm_mapping = plotting["emm_mapping"]

In [ ]:
year_tag = YEARS[0] if len(YEARS) == 1 else "all_years"
excel_file_path = f"efuels_supply_potentials_results_{year_tag}.xlsx"

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

import plotly.io as pio

pio.renderers.default = "colab"

### 1.2. Load Solved Network(s)

Scenario 1 is used as default for the analysis. Please change the number in `scenario_dir` to analyze a different scenario. The base year results are common for all the scenarios.

In [ ]:
from pathlib import Path
import pypsa

project_root = Path().resolve()
results_dir = project_root / "results"

networks = {}

LCOPT_SCENARIOS = {"05", "06"}

loaded_any = False

for sid in SCENARIO_IDS:
    for YEAR in YEARS:
        scenario_dir = results_dir / "scenarios" / f"scenario_{sid}"

        if not scenario_dir.exists():
            print(f"Skipping scenario_{sid}: directory not found")
            continue

        if sid in LCOPT_SCENARIOS:
            file_name = (
                f"elec_s_100_ec_lcopt_CCL-{RESOLUTION}_{RESOLUTION}_"
                f"{YEAR}_0.07_AB_0export.nc"
            )
        else:
            file_name = (
                f"elec_s_100_ec_lv1_CCL-{RESOLUTION}_{RESOLUTION}_"
                f"{YEAR}_0.07_AB_0export.nc"
            )

        file_path = scenario_dir / file_name
        key = f"scenario_{sid}_{YEAR}"

        if file_path.exists():
            networks[key] = pypsa.Network(file_path)
            print(f"Loaded {key}")
            loaded_any = True
        else:
            print(f"Missing: {file_path.name}")

if not loaded_any:
    raise RuntimeError(
        f"No networks loaded for YEAR={YEARS}, "
        f"SCENARIO_IDS={SCENARIO_IDS}, RESOLUTION={RESOLUTION}"
    )

print("\nLoaded networks:")
for k in networks:
    print(" -", k)

In [ ]:
from openpyxl import load_workbook, Workbook
from openpyxl.styles import Font, Alignment
from datetime import datetime
from pathlib import Path

excel_path = Path(excel_file_path)

# -----------------------------
# Load or create workbook
# -----------------------------
if excel_path.exists():
    wb = load_workbook(excel_path)
else:
    wb = Workbook()

sheet_name = f"Info_{YEAR}"

# -----------------------------
# Get or create Info_YEAR sheet
# -----------------------------
if sheet_name in wb.sheetnames:
    ws = wb[sheet_name]
else:
    ws = wb.create_sheet(sheet_name)

# -----------------------------
# Clear existing content
# -----------------------------
ws.delete_rows(1, ws.max_row)

# -----------------------------
# Header
# -----------------------------
ws.merge_cells("A1:D1")
ws["A1"] = "Project: Grid modelling to assess electrofuels supply potential"
ws["A1"].font = Font(bold=True, size=14)
ws["A1"].alignment = Alignment(horizontal="center", vertical="center")

# -----------------------------
# Metadata (MULTIPLE SCENARIO)
# -----------------------------
meta = [
    ("Notebook description", "Multiple scenario analysis"),
    ("Created on (UTC)", datetime.utcnow().strftime("%Y-%m-%d %H:%M:%SZ")),
    ("Scenarios", ", ".join(SCENARIO_IDS)),
    ("Temporal resolution", RESOLUTION),
    ("Planning horizon", YEAR),
]

row = 3
for label, value in meta:
    ws.cell(row=row, column=1, value=label).font = Font(bold=True)
    ws.cell(row=row, column=2, value=str(value))
    ws.row_dimensions[row].height = 14
    row += 1

# -----------------------------
# Column widths
# -----------------------------
ws.column_dimensions["A"].width = 32
ws.column_dimensions["B"].width = 100

# -----------------------------
# Save
# -----------------------------
wb.save(excel_file_path)
print(f"Info sheet '{sheet_name}' updated -> {excel_file_path}")

In [ ]:
grid_region_shapes_path = os.path.join(
    project_root, "data", "needs_grid_regions_aggregated.geojson"
)
state_shapes_path = os.path.join(project_root, "data", "gadm41_USA_1.json")

regions_onshore = gpd.read_file(grid_region_shapes_path)
emm_shapes_path = "./data/EIA_market_module_regions/EMM_regions.geojson"
regional_fees = pd.read_csv("./data/EIA_market_module_regions/regional_fees.csv")

In [ ]:
for network in networks.keys():
    attach_grid_region_to_buses(networks[network], grid_region_shapes_path)
    attach_state_to_buses(networks[network], state_shapes_path)
    attach_emm_region_to_buses(
        networks[network], emm_shapes_path, distance_crs="EPSG:3857"
    )

---

## 2. Total system cost

*A comparison of the total system cost across all analyzed scenarios. This is the key metric for overall economic performance.*

In [ ]:
all_system_costs = []

for name_tag, network in networks.items():
    df_yearly = compute_system_costs(
        network=network,
        rename_capex=rename_techs_capex,
        rename_opex=rename_techs_opex,
        name_tag=name_tag,
    )
    all_system_costs.append(df_yearly)

df_system_costs_all = pd.concat(all_system_costs, ignore_index=True)

df_system_costs_all["macro_category"] = df_system_costs_all.apply(
    lambda row: assign_macro_category(row, categories_capex, categories_opex), axis=1
)

# Plot CAPEX
plot_stacked_costs_by_year_plotly(
    df_system_costs_all,
    "Capital expenditure",
    tech_colors=rename_tech_colors,
    index="scenario",
)

# Plot OPEX
plot_stacked_costs_by_year_plotly(
    df_system_costs_all,
    "Operational expenditure",
    tech_colors=rename_tech_colors,
    index="scenario",
)
showfig()

In [ ]:
def process_costs_for_excel(df, sheet_name, cost_type, title):
    df_aggregated = (
        df[df["cost_type"] == cost_type]
        .groupby(["tech_label", "scenario"])
        .agg(
            {"cost_billion": "sum", "main_category": "first", "macro_category": "first"}
        )
        .reset_index()
    )

    df_pivoted = df_aggregated.pivot(
        index=["tech_label", "main_category", "macro_category"],
        columns="scenario",
        values=["cost_billion"],
    ).fillna(0)

    df_pivoted.columns = df_pivoted.columns.swaplevel(0, 1)
    df_pivoted = df_pivoted.sort_index(axis=1, level=[0, 1])
    df_pivoted = df_pivoted.droplevel(1, axis=1)
    df_pivoted.columns.name = None

    save_to_excel_with_formatting(
        df_pivoted, f"{sheet_name}", f"{title}", excel_file_path, freeze_pane="D3"
    )

In [ ]:
process_costs_for_excel(
    df_system_costs_all,
    "Total_System_Cost_CAPEX",
    "Capital expenditure",
    "Capital expenditure (Billion USD)",
)
process_costs_for_excel(
    df_system_costs_all,
    "Total_System_Cost_OPEX",
    "Operational expenditure",
    "Operational expenditure (Billion USD)",
)

In [ ]:
try:
    del df_original_costs, power_techs, all_original_costs
except:
    pass

# Rebuild cost data with links-only approach
all_original_costs = []
for name_tag, network in networks.items():
    df_costs = compute_links_only_costs(network, name_tag)
    all_original_costs.append(df_costs)

df_original_costs = pd.concat(all_original_costs, ignore_index=True)

# Identify power generation technologies
power_techs = identify_power_generation_technologies(
    rename_techs_capex, rename_techs_opex, categories_capex, categories_opex
)

# Process data for visualization
# Filter for technologies of interest only
mask = df_original_costs["tech_label"].isin(nice_names_power.keys())
df_power = df_original_costs[mask].copy()
# Apply mapping
df_power["tech_label"] = df_power["tech_label"].replace(nice_names_power)

# Aggregate technologies with same name (e.g. Offshore Wind AC+DC)
df_power_aggregated = df_power.groupby(
    ["tech_label", "cost_type", "year", "scenario"], as_index=False
).agg({"cost_billion": "sum"})

# Get mapped technology names
power_techs_mapped = set(df_power_aggregated["tech_label"].unique())

# Create charts
fig1 = plot_power_generation_details(
    df_power_aggregated,
    "Capital expenditure",
    power_techs_mapped,
    tech_colors=tech_power_color,
    nice_names=None,
    tech_order=tech_power_order,
    index="scenario",
)

fig2 = plot_power_generation_details(
    df_power_aggregated,
    "Operational expenditure",
    power_techs_mapped,
    tech_colors=tech_power_color,
    nice_names=None,
    tech_order=tech_power_order,
    index="scenario",
)

# Show charts
fig1.show()
fig2.show()

In [ ]:
def power_gen_capex_by_tech_and_scenario(df, sheet_name, cost_type, title):
    df_aggregated = (
        df[df["cost_type"] == cost_type]
        .groupby(["tech_label", "scenario"])
        .agg(
            {
                "cost_billion": "sum",
            }
        )
        .reset_index()
    )

    df_pivoted = df_aggregated.pivot(
        index="tech_label", columns="scenario", values="cost_billion"
    ).fillna(0)

    save_to_excel_with_formatting(
        df_pivoted, f"{sheet_name}", f"{title}", excel_file_path, freeze_pane="B3"
    )

In [ ]:
power_gen_capex_by_tech_and_scenario(
    df_power_aggregated,
    "Power_Gen_CAPEX",
    "Capital expenditure",
    "Power Generation - Capital Expenditure (Billion USD)",
)
power_gen_capex_by_tech_and_scenario(
    df_power_aggregated,
    "Power_Gen_OPEX",
    "Operational expenditure",
    "Power Generation - Operational Expenditure (Billion USD)",
)

In [ ]:
total_tax_credit_table = {}
for name, network in networks.items():
    tax_credit_df = compute_power_opex_with_tax_credits(network, name)
    tax_credit_df.tech_label = tax_credit_df["tech_label"].replace(nice_names)
    tax_credit_df.tech_label = tax_credit_df["tech_label"].replace(nice_names_power)
    total_tax_credit_table[name] = tax_credit_df

total_tax_credit_df = pd.concat(total_tax_credit_table, ignore_index=True)
total_tax_credit_df["cost_type"] = "Operational expenditure"

In [ ]:
save_to_excel_with_formatting(
    total_tax_credit_df,
    "Power_Gen_OPEX_with_Tax_Credits",
    "Power Generation - Operational Expenditure with Tax Credits (Billion USD)",
    excel_file_path,
    freeze_pane="B3",
)

In [ ]:
techs_to_show = [
    "Nuclear",
    "Geothermal",
    "Biomass",
    "Biomass CHP",
    "Utility-scale solar",
    "Onshore wind",
    "Offshore wind",
]
fig = plot_tax_credit_cluster_bars(
    total_tax_credit_df.query("tech_label in @techs_to_show"),
    tech_power_color,
    nice_names_power,
    title="Power Generation: OPEX With vs. Without Tax Credits",
    cost_type="OPEX",
    index="scenario",
)
fig.show()

In [ ]:
cc_energy_cost_dfs = {}
for network_name, net in networks.items():
    # Extract last block of 4-digit year
    match = re.findall(r"\d{4}", network_name)
    year = match[-1] if match else network_name

    co2_cap = compute_captured_co2_by_tech_from_network(net)
    df = compute_cc_energy_costs(net, year, co2_cap)
    df = df.set_index("Technology")  # Set Technology as index
    cc_energy_cost_dfs[network_name] = df

df_cc_energy_costs_all = pd.concat(
    cc_energy_cost_dfs, axis=1, keys=cc_energy_cost_dfs.keys()
)
save_to_excel_with_formatting(
    df_cc_energy_costs_all,
    "CC_Energy_Costs",
    "Carbon Capture Energy Costs (MWh/ton CO2)",
    excel_file_path,
    freeze_pane="B4",
)

In [ ]:
electrolyzer_carriers = {"Alkaline electrolyzer large", "PEM electrolyzer", "SOEC"}

fig = plot_tax_credit_cluster_bars(
    total_tax_credit_df.query("tech_label in @electrolyzer_carriers"),
    tech_colors,
    rename_techs_opex,
    title="Hydrogen production OPEX With vs. Without Tax Credits",
    unit="million",
    cost_type="OPEX",
    index="scenario",
)
fig.show()

In [ ]:
total_capex_tax_credit_table = {}

for name, network in networks.items():
    capex_tax_credit_df = compute_power_capex_with_tax_credits(network, name)
    total_capex_tax_credit_table[name] = capex_tax_credit_df

total_capex_tax_credit_df = pd.concat(
    total_capex_tax_credit_table.values(), ignore_index=True
)

plot_tax_credit_cluster_bars(
    total_capex_tax_credit_df.query("tech_label == 'battery'"),
    tech_colors,
    rename_techs_opex,
    title="Battery: CAPEX With vs. Without Tax Credits",
    unit="million",
    cost_type="CAPEX",
)

In [ ]:
df_h2_efuels_results = create_h2_efuels_analysis(networks, index="scenario")

In [ ]:
def h2_efuels_costs_for_excel(df, sheet_name, cost_type, title):
    df_aggregated = (
        df[df["cost_type"] == cost_type]
        .groupby(["tech_label", "scenario"])
        .agg(
            {
                "cost_billion": "sum",
            }
        )
        .reset_index()
    )

    df_aggregated_pivoted = df_aggregated.pivot(
        index="tech_label", columns="scenario", values="cost_billion"
    ).fillna(0)

    df_aggregated_pivoted
    save_to_excel_with_formatting(
        df_aggregated_pivoted,
        f"{sheet_name}",
        f"{title}",
        excel_file_path,
        freeze_pane="B3",
    )

In [ ]:
h2_efuels_costs_for_excel(
    df_h2_efuels_results,
    "H2_Efuels_CAPEX",
    "Capital expenditure",
    "H2 & e-fuels Technologies - Capital Expenditure (Billion USD)",
)
h2_efuels_costs_for_excel(
    df_h2_efuels_results,
    "H2_Efuels_OPEX",
    "Operational expenditure",
    "H2 & e-fuels Technologies - Operational Expenditure (Billion USD)",
)

---

## 3. Electricity demand

*This section illustrates results concerning electricity demand across the different sectors in the model, with a specific focus on data centers.*

### 3.1. Electricity demand by service
*This section reports results for electricity demand related to the different sectors, providing a comparison against the NREL Electrification Futures Studies (EFS) scenarios.*

In [ ]:
# Industrial processes consuming AC electricity
target_processes = [
    "SMR CC",
    "Haber-Bosch",
    "ethanol from starch",
    "ethanol from starch CC",
    "DRI",
    "DRI CC",
    "DRI H2",
    "BF-BOF",
    "BF-BOF CC",
    "EAF",
    "dry clinker",
    "cement finishing",
    "dry clinker CC",
]

# Static and dynamic loads
static_load_carriers = [
    "rail transport electricity",
    "agriculture electricity",
    "industry electricity",
]
dynamic_load_carriers = [
    "AC",
    "services electricity",
    "land transport EV",
    "data center",
]

demand = pd.DataFrame(
    columns=networks.keys(),
    index=dynamic_load_carriers + static_load_carriers + ["total demand"],
)
demand.index.name = "Load type (TWh)"

for name, n in networks.items():
    nhours = n.snapshot_weightings.objective.sum()

    # Static loads
    static_totals = (
        n.loads.groupby("carrier").sum().p_set.reindex(static_load_carriers).fillna(0)
    )
    static_load_twh = static_totals.sum() * nhours / 1e6

    # Industrial AC
    process_links = n.links[n.links.carrier.isin(target_processes)]
    ac_input_links = process_links[
        process_links.bus0.map(n.buses.carrier) == "AC"
    ].index
    ind_ac_profile = n.links_t.p0[ac_input_links].sum(axis=1)
    ind_ac_twh = (ind_ac_profile * n.snapshot_weightings.objective).sum() / 1e6

    # Non-industrial AC
    ac_loads = n.loads[n.loads.carrier == "AC"]
    industrial_ac_buses = n.links.loc[ac_input_links, "bus0"].unique()
    ac_non_ind_idx = ac_loads[~ac_loads.bus.isin(industrial_ac_buses)].index
    ac_profile = n.loads_t.p_set[
        ac_non_ind_idx.intersection(n.loads_t.p_set.columns)
    ].sum(axis=1)
    ac_twh = (ac_profile * n.snapshot_weightings.objective).sum() / 1e6 - ind_ac_twh

    # Services and EVs
    serv_idx = [
        i
        for i in n.loads[n.loads.carrier == "services electricity"].index
        if i in n.loads_t.p_set.columns
    ]
    ev_idx = [
        i
        for i in n.loads[n.loads.carrier == "land transport EV"].index
        if i in n.loads_t.p_set.columns
    ]
    serv_profile = n.loads_t.p_set[serv_idx].sum(axis=1) if serv_idx else 0
    ev_profile = n.loads_t.p_set[ev_idx].sum(axis=1) if ev_idx else 0
    serv_twh = (serv_profile * n.snapshot_weightings.objective).sum() / 1e6
    ev_twh = (ev_profile * n.snapshot_weightings.objective).sum() / 1e6

    # Data centers
    data_center_p_set_sum = n.loads.loc[n.loads.carrier == "data center", "p_set"].sum()
    data_center_twh = data_center_p_set_sum * nhours / 1e6

    # Other electricity (included in industrial consumption)
    other_idx = [
        i
        for i in n.loads[n.loads.carrier == "other electricity"].index
        if i in n.loads_t.p_set.columns
    ]
    other_profile = n.loads_t.p_set[other_idx].sum(axis=1) if other_idx else 0
    other_twh = (other_profile * n.snapshot_weightings.objective).sum() / 1e6

    industry_static_twh = static_totals.get("industry electricity", 0) * nhours / 1e6
    industry_elec_twh = industry_static_twh + ind_ac_twh + other_twh

    demand.loc["rail transport electricity", name] = (
        static_totals.get("rail transport electricity", 0) * nhours / 1e6
    )
    demand.loc["agriculture electricity", name] = (
        static_totals.get("agriculture electricity", 0) * nhours / 1e6
    )
    demand.loc["industry electricity", name] = industry_elec_twh
    demand.loc["AC", name] = ac_twh
    demand.loc["services electricity", name] = serv_twh
    demand.loc["land transport EV", name] = ev_twh
    demand.loc["data center", name] = data_center_twh
    demand.loc["total demand", name] = (
        static_load_twh
        + ac_twh
        + ind_ac_twh
        + serv_twh
        + ev_twh
        + data_center_twh
        + other_twh
    )
    demand.loc["total demand no data center", name] = (
        static_load_twh + ac_twh + ind_ac_twh + serv_twh + ev_twh + other_twh
    )

In [ ]:
# NREL reference values (2023, 2030, 2035, 2040) - unchanged
nrel_data_medium = {
    "AC (residential + some industrial processes)": [1360.81, 1359.91, 1392.19],
    "services electricity": [1381.11, 1398.69, 1579.71],
    "land transport EV": [13.10, 275.97, 714.41],
    "rail transport electricity": [7.29, 7.64, 8.29],
    "agriculture electricity": [np.nan, np.nan, np.nan],
    "industry electricity": [1036.81, 1109.69, 1213.94],
    "data center": [np.nan, np.nan, np.nan],
    "total demand (incl. data centers)": [3799.12, 4293.30, 5072.86],
    "total demand (excl. data centers)": [3799.12, 4293.30, 5072.86],
}

nrel_data_high = {
    "AC (residential + some industrial processes)": [1360.81, 1395.21, 1529.51],
    "services electricity": [1381.11, 1470.10, 1673.71],
    "land transport EV": [13.10, 349.15, 1039.76],
    "rail transport electricity": [7.29, 9.96, 12.71],
    "agriculture electricity": [np.nan, np.nan, np.nan],
    "industry electricity": [1036.81, 1152.90, 1286.24],
    "data center": [np.nan, np.nan, np.nan],
    "total demand (incl. data centers)": [3799.12, 4377.32, 6713.68],
    "total demand (excl. data centers)": [3799.12, 4377.32, 6713.68],
}

# Mapping PyPSA names to NREL names - unchanged
row_rename_map = {
    "AC": "AC (residential + some industrial processes)",
    "total demand": "total demand (incl. data centers)",
    "total demand no data center": "total demand (excl. data centers)",
}

# Collect all networks with their year and scenario
network_details = []
for key in networks.keys():
    match = re.search(r"(?:scenario_(\d{2})|Base)_(\d{4})", key)
    if match:
        if match.group(1):
            scenario = f"scenario_{match.group(1)}"
        else:
            scenario = "Base"
        year = int(match.group(2))
        network_details.append((year, scenario, key))
    else:
        print(
            f"Warning: Skipping network '{key}' - does not match expected format (e.g., 'scenario_01_2030' or 'Base_2023')."
        )

# Group by year
from collections import defaultdict

networks_by_year = defaultdict(list)
for year, scenario, key in network_details:
    networks_by_year[year].append((scenario, key))

# Prepare the comparison table
rows = list(nrel_data_medium.keys())  # Same for both medium and high
years = sorted(networks_by_year.keys())
scenarios_per_year = {year: [s for s, _ in networks_by_year[year]] for year in years}

# Get all unique scenarios across years, with "Base" first
all_scenarios = sorted(
    set(
        scenario for scenarios in scenarios_per_year.values() for scenario in scenarios
    ),
    key=lambda x: (x != "Base", x),
)

# Create MultiIndex columns: (scenario, year, metric)
columns = []
for scenario in all_scenarios:
    for year in years:
        if scenario in scenarios_per_year[year]:
            columns.extend(
                [
                    (scenario, year, "PyPSA-Earth"),
                    (scenario, year, "NREL"),
                    (scenario, year, "diff %"),
                ]
            )
comparison_df = pd.DataFrame(index=rows, columns=pd.MultiIndex.from_tuples(columns))

# Populate the comparison values
for year in years:
    nrel_years = [2023, 2030, 2035, 2040]
    year_idx = nrel_years.index(year) if year in nrel_years else None

    for scenario, net_key in networks_by_year[year]:
        # Select NREL data based on scenario
        if scenario in [
            "scenario_05",
            "scenario_06",
        ]:  # Updated to match full scenario names
            nrel_data = nrel_data_high
        else:
            nrel_data = nrel_data_medium

        for row in rows:
            pypsa_row = {v: k for k, v in row_rename_map.items()}.get(row, row)
            pypsa_val = (
                demand.at[pypsa_row, net_key] if pypsa_row in demand.index else np.nan
            )

            if year_idx is not None:
                nrel_vals = nrel_data.get(row, [np.nan] * len(nrel_years))
                nrel_val = nrel_vals[year_idx] if len(nrel_vals) > year_idx else np.nan
            else:
                nrel_val = np.nan

            comparison_df.at[row, (scenario, year, "PyPSA-Earth")] = pypsa_val
            comparison_df.at[row, (scenario, year, "NREL")] = nrel_val

            if pd.notna(pypsa_val) and pd.notna(nrel_val) and nrel_val != 0:
                diff = 100 * (pypsa_val - nrel_val) / nrel_val
            else:
                diff = np.nan
            comparison_df.at[row, (scenario, year, "diff %")] = diff


# Style function to highlight significant differences - updated for new structure
def highlight_diff(val):
    if pd.isna(val):
        return ""
    if val > 10:
        return "background-color: #ffcccc"  # light red
    elif val < -10:
        return "background-color: #cce5ff"  # light blue
    else:
        return "background-color: #d4edda"  # light green


# Apply styling to the diff columns
styled = comparison_df.style
for scenario in all_scenarios:
    for year in years:
        if scenario in scenarios_per_year[year]:
            styled = styled.applymap(
                highlight_diff, subset=pd.IndexSlice[:, (scenario, year, "diff %")]
            )
styled = styled.format(na_rep="N/A", precision=2)

# Display the final table
print("\nElectric load by sector (TWh/year) - Comparison across all scenarios")
display(styled)

In [ ]:
save_to_excel_with_formatting(
    comparison_df.fillna(0),
    "Electric load by sector",
    "Demand Summary (TWh/year)",
    excel_file_path,
    freeze_pane="B5",
)

In [ ]:
y_max = 0
demand_profiles_incl = {}
demand_profiles_excl = {}

for idx, (name, network) in enumerate(networks.items()):
    n = networks[name]
    nhours = n.snapshot_weightings.objective.sum()

    # Dynamic loads: AC, services, EV, other electricity
    dynamic_loads = n.loads[
        n.loads.carrier.isin(
            ["AC", "services electricity", "land transport EV", "other electricity"]
        )
    ]
    dynamic_idx = dynamic_loads.index.intersection(n.loads_t.p_set.columns)
    dyn_profile = n.loads_t.p_set[dynamic_idx].sum(axis=1)

    # Static loads: rail, agriculture, industry
    static_carriers = [
        "rail transport electricity",
        "agriculture electricity",
        "industry electricity",
    ]
    static_load = n.loads[n.loads.carrier.isin(static_carriers)]
    static_sum = static_load.groupby("carrier").sum()["p_set"].sum()  # MW
    static_profile = pd.Series(static_sum, index=n.snapshots)

    # Data centers (flat profile if present)
    if "data center" in n.loads.carrier.values:
        dc_sum = n.loads[n.loads.carrier == "data center"]["p_set"].sum()
        dc_profile = pd.Series(dc_sum, index=n.snapshots)
    else:
        dc_profile = 0

    # Total profiles (in GW)
    profile_excl = (dyn_profile + static_profile) / 1000
    profile_incl = (dyn_profile + static_profile + dc_profile) / 1000

    demand_profiles_excl[name] = profile_excl
    demand_profiles_incl[name] = profile_incl

    y_max = max(y_max, profile_incl.max())

y_max *= 1.05  # add 5% margin

# 3. Plot profiles for each year
fig, axes = plt.subplots(
    nrows=len(networks), figsize=(12, 4 * len(networks)), sharey=True
)

for ax, (name, network) in zip(axes, networks.items()):
    demand_profiles_excl[name].plot(
        ax=ax, linewidth=0.5, color="tab:blue", label="Excl. data centers"
    )
    demand_profiles_incl[name].plot(
        ax=ax, linewidth=0.5, color="tab:orange", label="Incl. data centers"
    )

    ax.set_title(f"Electricity demand profile - {name}")
    ax.set_xlabel("Time")
    ax.set_ylabel("Demand (GW)")
    ax.set_ylim(0, y_max)
    ax.legend(loc="upper right")

plt.tight_layout()
showfig()

### 3.2. Data Center Loads/Demands
*Isolating and visualizing the specific demand profile of data centers to understand their impact on the system.*

In [ ]:
# Initialize a dictionary to store demand by grid region for each scenario
data_center_demand_by_region = {}

# Compute data center load for each scenario
data_center_load = {}
for key, net in networks.items():
    data_center_load[key] = compute_data_center_load(net)

# Process each scenario to extract demand by grid region
for key, df in data_center_load.items():
    # Group by grid_region and sum p_set, convert to GW
    demand_by_grid_region = df.groupby("grid_region")["p_set"].sum().div(1e3)
    # Filter for positive demand
    demand_by_grid_region = demand_by_grid_region[demand_by_grid_region > 0]

    if demand_by_grid_region.empty:
        print(f"No demand data for {key}\n")
        continue

    # Store in the dictionary
    data_center_demand_by_region[key] = demand_by_grid_region

# Now, create a single DataFrame with grid_region as index and scenarios as columns
# First, collect all unique grid regions across scenarios
all_grid_regions = set()
for df in data_center_demand_by_region.values():
    all_grid_regions.update(df.index)
all_grid_regions = sorted(list(all_grid_regions))

# Create a DataFrame with grid_region as index and scenarios as columns
merged_df = pd.DataFrame(index=all_grid_regions)

for scenario, df in data_center_demand_by_region.items():
    merged_df[scenario] = df.reindex(all_grid_regions).fillna(0)

# Display the merged DataFrame
print("\nMerged Data Center Demand by Grid Region (GW)\n")
display(merged_df.style.format(precision=2, na_rep="0.00"))

# Save to Excel using the existing function (adjusted for single-level columns)
save_to_excel_with_formatting(
    merged_df,
    "Data_Center_Demand_Grid_Region",
    "Data Center Demand by Grid Region (GW)",
    excel_file_path,
)

In [ ]:
data_center_load = {}
for key, net in networks.items():
    data_center_load[key] = compute_data_center_load(net)

# Compute the maximum value to normalize the y-axis
max_val = 0
for df in data_center_load.values():
    demand_by_grid_region = df.groupby("grid_region")["p_set"].sum().div(1e3)
    demand_by_grid_region = demand_by_grid_region[demand_by_grid_region > 0]
    if not demand_by_grid_region.empty:
        max_val = max(max_val, demand_by_grid_region.max())

ymax = max(max_val * 1.05, 1)

# Main loop to generate plots
for key, df in data_center_load.items():
    demand_by_grid_region = df.groupby("grid_region")["p_set"].sum().div(1e3)
    demand_by_grid_region = demand_by_grid_region[demand_by_grid_region > 0]

    # Extract year from key, assuming format like "scenario_01_2030"
    # year_match = re.search(r"\d{4}", key)
    # year = year_match.group() if year_match else "Unknown Year"

    if demand_by_grid_region.empty:
        print(f"No demand data for {key}\n")
        continue

    ax = demand_by_grid_region.plot(
        kind="bar",
        title=f"Data center electricity demand by Grid region - {key}",
        ylabel="Data centers demand (GW)",
        xlabel="Grid region",
        figsize=(8, 4),
        legend=False,
    )

    # Add text labels on top of bars
    for i, value in enumerate(demand_by_grid_region.values):
        ax.text(
            i,
            value + ymax * 0.01,  # Slightly above the bar
            f"{value:.1f}",  # Format with 1 decimal place
            ha="center",
            va="bottom",
            fontsize=9,
            rotation=0,
        )

    ax.set_ylim(0, ymax)
    plt.xticks(rotation=-30)
    # plt.tight_layout()
    showfig()

In [ ]:
# Initialize a dictionary to store demand by state for each scenario
data_center_demand_by_state = {}

# Compute data center load for each scenario
data_center_load = {}
for key, net in networks.items():
    data_center_load[key] = compute_data_center_load(net)

# Process each scenario to extract demand by state
for key, df in data_center_load.items():
    # Group by state and sum p_set, convert to GW
    demand_by_state = df.groupby("state")["p_set"].sum().div(1e3)
    # Filter for positive demand
    demand_by_state = demand_by_state[demand_by_state > 0]

    if demand_by_state.empty:
        print(f"No demand data for {key}\n")
        continue

    # Store in the dictionary
    data_center_demand_by_state[key] = demand_by_state

# Now, create a single DataFrame with state as index and scenarios as columns
# First, collect all unique states across scenarios
all_states = set()
for df in data_center_demand_by_state.values():
    all_states.update(df.index)
all_states = sorted(list(all_states))

# Create a DataFrame with state as index and scenarios as columns
merged_df = pd.DataFrame(index=all_states)

for scenario, df in data_center_demand_by_state.items():
    merged_df[scenario] = df.reindex(all_states).fillna(0)

# Display the merged DataFrame
print("\nMerged Data Center Demand by State (GW)\n")
display(merged_df.style.format(precision=2, na_rep="0.00"))

# Save to Excel using the existing function (adjusted for single-level columns)
save_to_excel_with_formatting(
    merged_df,
    "Data_Center_Demand_State",
    "Data Center Demand by State (GW)",
    excel_file_path,
)

print(f"\nExported merged data center demand by state to {excel_file_path}\n")

In [ ]:
for key, df in data_center_load.items():
    demand_by_state = df.groupby("state")["p_set"].sum().div(1e3)
    demand_by_state = demand_by_state[demand_by_state > 0]

    if demand_by_state.empty:
        print(f"\nNo demand data for {key}\n")
        continue

    # Extract year from key assuming format like "scenario_01_2030"
    year_match = re.search(r"\d{4}", key)
    year = year_match.group() if year_match else "Unknown Year"

    ax = demand_by_state.plot(
        kind="bar",
        title=f"Data center electricity demand by State - {key}",
        ylabel="Data centers demand (GW)",
        xlabel="State",
        figsize=(10, 4),
        legend=False,
    )

    # Add text labels on top of bars
    for i, value in enumerate(demand_by_state.values):
        ax.text(
            i,
            value + max_val * 0.01,  # Slightly above the bar
            f"{value:.1f}",
            ha="center",
            va="bottom",
            fontsize=9,
        )

    ax.set_ylim(0, max_val * 1.05)
    plt.xticks(rotation=0)
    plt.tight_layout()
    showfig()

---

## 4. Capacity Analysis: What Was Built?

*Here, we analyze the optimal installed capacities of generation, storage, and conversion technologies as determined by the model.*

### 4.1. Map: Total Installed Electricity Capacity
*A map showing the total installed capacity (in GW) for each electricity generation carrier (e.g., Solar, Onshore Wind, Offshore Wind) at different locations.*

In [ ]:
for key, n in networks.items():
    plot_network_generation_and_transmission(
        n, key, tech_colors, nice_names, regions_onshore, title_year=False
    )

In [ ]:
df_capacity = compute_installed_capacity_by_carrier(
    networks, nice_names=nice_names, column_year=False
)

In [ ]:
save_to_excel_with_formatting(
    df_capacity,
    "Installed capacity by Tech",
    "Installed_Capacity (GW)",
    excel_file_path,
)

In [ ]:
# Invert nice_names so we can map pretty name to original PyPSA carrier
inverse_nice = {v: k for k, v in nice_names.items()}


def color_for_pretty(pretty_name: str) -> str:
    """Get the color for a pretty name, using tech_colors based on the original key"""
    orig = inverse_nice.get(pretty_name, pretty_name)
    return tech_colors.get(orig, "gray")


carrier_capacity_df = compute_installed_capacity_by_carrier(
    networks, display_result=False, column_year=False
)

# Preferred order (matching the summary table)
preferred_order = [
    "Coal",
    "Gas CCGT",
    "Gas OCGT",
    "Gas CHP",
    "Oil",
    "Nuclear",
    "Biomass",
    "Biomass CHP",
    "Geothermal",
    "Conventional hydro",
    "Run-of-River hydro",
    "Pumped hydro storage",
    "Onshore wind",
    "Offshore wind",
    "Utility-scale solar",
    "Rooftop solar",
    "CSP",
    "Battery",
]

# Convert pretty names → original carrier keys, keep only available ones
available = carrier_capacity_df.index.tolist()
preferred_index = [
    inverse_nice.get(p, p)
    for p in preferred_order
    if inverse_nice.get(p, p) in available
]
rest = [c for c in available if c not in preferred_index]
ordered_index = preferred_index + rest

# Reorder DataFrame rows
carrier_capacity_df = carrier_capacity_df.loc[ordered_index]

# Get the tech order and colors
color_list = [tech_colors.get(carrier, "gray") for carrier in carrier_capacity_df.index]

# Create the Plotly stacked bar chart
fig = go.Figure()

for carrier, color in zip(carrier_capacity_df.index, color_list):
    fig.add_trace(
        go.Bar(
            name=nice_names.get(carrier, carrier),  # Pretty name
            x=carrier_capacity_df.columns.astype(str),  # Years
            y=carrier_capacity_df.loc[carrier],
            marker_color=color,
            width=0.6,
            hovertemplate=f"%{{x}}<br>{nice_names.get(carrier, carrier)}: %{{y:.2f}}GW <extra></extra>",
        )
    )

fig.update_layout(
    barmode="stack",
    title=dict(
        text="Installed capacity by technology",
        x=0.5,
        xanchor="center",
        font=dict(size=24),
    ),
    xaxis_title="Year (-)",
    yaxis_title="Installed capacity (GW)",
    legend_title="Technology",
    template="plotly_white",
    width=1400,
    height=700,
    xaxis=dict(
        title="Scenarios",
        tickangle=90,
        showline=True,
        linecolor="black",
    ),
    yaxis=dict(
        title="Installed capacity (GW)",
        showline=True,
        linecolor="black",
    ),
    font=dict(size=16),
)
fig.add_shape(
    type="rect",
    xref="paper",
    yref="paper",
    x0=0,
    y0=0,
    x1=1,
    y1=1,
    line=dict(color="black", width=1),
    layer="below",
)

fig.show()

In [ ]:
# Prepare data structures
all_data = defaultdict(dict)  # {grid_region: {(scenario, year, metric): value}}
scenarios_years = set()  # To track all (scenario, year) combinations

# Loop over all networks
for network_name, n in networks.items():
    # Extract scenario and year from network_name (e.g., "scenario_01_2030" -> scenario="scenario_01", year=2030)
    match = re.search(r"(?:scenario_(\d{2})|Base)_(\d{4})", network_name)
    if match:
        if match.group(1):
            scenario = f"scenario_{match.group(1)}"
        else:
            scenario = "Base"
        year = int(match.group(2))
    else:
        print(
            f"Warning: Skipping network '{network_name}' - does not match expected format (e.g., 'scenario_01_2030' or 'Base_2023')."
        )
        continue

    # Get line expansion data (grouped by grid_region)
    df = compute_line_expansion_capacity(
        n
    )[
        0
    ]  # Assumes returns DF with index="grid_region", columns=["s_nom", "s_nom_opt"] in MW

    # Convert to GW
    df = df / 1000  # MW to GW

    # Store data for each grid_region
    for grid_region in df.index:
        all_data[grid_region][(scenario, year, "s_nom")] = df.at[grid_region, "s_nom"]
        all_data[grid_region][(scenario, year, "s_nom_opt")] = df.at[
            grid_region, "s_nom_opt"
        ]

    # Track scenarios and years
    scenarios_years.add((scenario, year))

# Build the MultiIndex DataFrame
# Get all unique grid_regions
grid_regions = sorted(all_data.keys())

# Get all unique (scenario, year) combinations, sorted by scenario then year
sorted_scenarios_years = sorted(scenarios_years, key=lambda x: (x[0], x[1]))

# Create columns: (scenario, year, metric)
columns = []
for scenario, year in sorted_scenarios_years:
    columns.extend([(scenario, year, "s_nom"), (scenario, year, "s_nom_opt")])

# Create the DataFrame
expansion_df = pd.DataFrame(
    index=grid_regions, columns=pd.MultiIndex.from_tuples(columns)
)

# Populate the DataFrame
for grid_region, data_dict in all_data.items():
    for key, value in data_dict.items():
        expansion_df.at[grid_region, key] = value

# Rename columns for clarity (optional, but matches original intent)
expansion_df.columns = pd.MultiIndex.from_tuples(
    [
        (
            scenario,
            year,
            "Transmission capacity (GW) - 2023"
            if metric == "s_nom"
            else f"Expanded transmission capacity (GW) - {year}",
        )
        for scenario, year, metric in expansion_df.columns
    ]
)

save_to_excel_with_formatting(
    expansion_df,
    "Transmission Expansion Grid",
    "Transmission Line Expansion Capacity (GW) - Comparison across all scenarios",
    excel_file_path,
    freeze_pane="B5",
)
# Style and display the final DataFrame
styled_df = expansion_df.style.format("{:.2f}", na_rep="N/A")
print("\nTransmission Line Expansion Capacity (GW) - Comparison across all scenarios")
display(styled_df)

In [ ]:
for network_name, n in networks.items():
    line_expan_cap = compute_line_expansion_capacity(n)[0]

    line_expan_cap = line_expan_cap.rename(
        columns={"s_nom": "Installed (2023)", "s_nom_opt": "Expanded"}
    )

    line_expan_cap.plot(
        kind="bar",
        title=f"Transmission capacity by Grid Region - {network_name}",
        ylabel="Capacity (GW)",
        xlabel="Grid regions",
        figsize=(10, 5),
        legend=True,
    )

    plt.xticks(rotation=45)
    plt.tight_layout()
    showfig()

In [ ]:
# Prepare data structures
all_data = defaultdict(dict)  # {state: {(scenario, year, metric): value}}
scenarios_years = set()  # To track all (scenario, year) combinations

# Loop over all networks
for network_name, n in networks.items():
    # Extract scenario and year from network_name (e.g., "scenario_01_2030" -> scenario="scenario_01", year=2030)
    match = re.search(r"(?:scenario_(\d{2})|Base)_(\d{4})", network_name)
    if match:
        if match.group(1):
            scenario = f"scenario_{match.group(1)}"
        else:
            scenario = "Base"
        year = int(match.group(2))
    else:
        print(
            f"Warning: Skipping network '{network_name}' - does not match expected format (e.g., 'scenario_01_2030' or 'Base_2023')."
        )
        continue

    # Get line expansion data (grouped by state)
    df = compute_line_expansion_capacity(n)[
        1
    ]  # Assumes returns DF with index="state", columns=["s_nom", "s_nom_opt"] in MW

    # Convert to GW
    df = df / 1000  # MW to GW

    # Store data for each state
    for state in df.index:
        all_data[state][(scenario, year, "s_nom")] = df.at[state, "s_nom"]
        all_data[state][(scenario, year, "s_nom_opt")] = df.at[state, "s_nom_opt"]

    # Track scenarios and years
    scenarios_years.add((scenario, year))

# Build the MultiIndex DataFrame
# Get all unique states
states = sorted(all_data.keys())

# Get all unique (scenario, year) combinations, sorted by scenario then year
sorted_scenarios_years = sorted(scenarios_years, key=lambda x: (x[0], x[1]))

# Create columns: (scenario, year, metric)
columns = []
for scenario, year in sorted_scenarios_years:
    columns.extend([(scenario, year, "s_nom"), (scenario, year, "s_nom_opt")])

# Create the DataFrame
expansion_df = pd.DataFrame(index=states, columns=pd.MultiIndex.from_tuples(columns))

# Populate the DataFrame
for state, data_dict in all_data.items():
    for key, value in data_dict.items():
        expansion_df.at[state, key] = value

# Rename columns for clarity (optional, but matches original intent)
expansion_df.columns = pd.MultiIndex.from_tuples(
    [
        (
            scenario,
            year,
            "Transmission capacity (GW) - 2023"
            if metric == "s_nom"
            else f"Expanded transmission capacity (GW) - {year}",
        )
        for scenario, year, metric in expansion_df.columns
    ]
)

save_to_excel_with_formatting(
    expansion_df,
    "Transmission Expansion State",
    "Transmission Line Expansion Capacity (GW) - Comparison across all scenarios (Grouped by State)",
    excel_file_path,
    freeze_pane="B5",
)

# Style and display the final DataFrame
styled_df = expansion_df.style.format("{:.2f}", na_rep="N/A")
print(
    "\nTransmission Line Expansion Capacity (GW) - Comparison across all scenarios (Grouped by State)"
)
display(styled_df)

In [ ]:
for network_name, n in networks.items():
    line_expan_cap = compute_line_expansion_capacity(n)[1]

    line_expan_cap = line_expan_cap.rename(
        columns={"s_nom": "Installed (2023)", "s_nom_opt": "Expanded"}
    )

    line_expan_cap.plot(
        kind="bar",
        title=f"Transmission capacity by State - {network_name}",
        ylabel="Capacity (GW)",
        xlabel="States",
        figsize=(18, 6),
        legend=True,
    )

    plt.xticks(rotation=45)
    plt.tight_layout()
    showfig()

### 4.2. Hydrogen production capacity
*This section reports results about installed hydrogen production capacity of the different electrolyzer technologies (e.g., Alkaline, PEM, SOEC).*

In [ ]:
for idx, network in enumerate(networks.keys()):
    plot_h2_capacities_map(
        networks[network], network, tech_colors, nice_names, regions_onshore
    )

In [ ]:
h2_cap_by_network = {}
max_n_states = 0
max_capacity = 0

for name, network in networks.items():
    h2_cap = (
        compute_h2_capacities(network)
        .groupby("state")[["Alkaline electrolyzer large", "PEM electrolyzer", "SOEC"]]
        .sum()
    )

    filtered = h2_cap[h2_cap.sum(axis=1) > 1]

    h2_cap_by_network[name] = filtered

    if not filtered.empty:
        max_n_states = max(max_n_states, filtered.shape[0])
        max_capacity = max(max_capacity, filtered.sum(axis=1).max())

for name, filtered in h2_cap_by_network.items():
    if filtered.empty:
        print(f"\nSkipping {name}: no States with capacity > 1 MW\n")
        continue

    plot_h2_capacities_by_state(filtered, name, max_capacity, max_n_states)

In [ ]:
h2_cap_by_network = {}
max_n_grid_regions = 0
max_capacity = 0

for name, network in networks.items():
    h2_cap = (
        compute_h2_capacities(network)
        .groupby("grid_region")[
            ["Alkaline electrolyzer large", "PEM electrolyzer", "SOEC"]
        ]
        .sum()
    )

    filtered = h2_cap[h2_cap.sum(axis=1) > 1]

    h2_cap_by_network[name] = filtered

    if not filtered.empty:
        max_n_grid_regions = max(max_n_grid_regions, filtered.shape[0])
        max_capacity = max(max_capacity, filtered.sum(axis=1).max())

for name, filtered in h2_cap_by_network.items():
    if filtered.empty:
        print(f"\nSkipping {name}: no Grid Regions with capacity > 1 MW\n")
        continue

    plot_h2_capacities_by_grid_region(filtered, name, max_capacity, max_n_grid_regions)

In [ ]:
# First, collect all unique states across all scenarios
all_states = set()
for network in networks.keys():
    h2_state_cap = (
        compute_h2_capacities(networks[network])
        .groupby(["state"])[["Alkaline electrolyzer large", "PEM electrolyzer", "SOEC"]]
        .sum()
    )
    all_states.update(h2_state_cap.index)

all_states = sorted(list(all_states))  # Sort for consistency

all_h2_state_caps = []

for idx, network in enumerate(networks.keys()):
    h2_state_cap = (
        compute_h2_capacities(networks[network])
        .groupby(["state"])[["Alkaline electrolyzer large", "PEM electrolyzer", "SOEC"]]
        .sum()
    )

    print(
        f"\nInstalled electrolyzer capacity by State - {network} (only States ≥ 10 kW)\n"
    )

    # Transpose and filter states with capacity >= 0.01 MW (10 kW) for display only
    df_display = h2_state_cap.T
    df_display = df_display.loc[:, (df_display >= 0.01).any(axis=0)]
    df_display.columns.name = "State"
    df_display.index.name = None

    display(df_display.style.format("{:.1e}"))

    # For merging: Create a full DataFrame with all states, filling missing with 0
    df_full = pd.DataFrame(
        index=all_states,
        columns=["Alkaline electrolyzer large", "PEM electrolyzer", "SOEC"],
    ).fillna(0)
    # Update with actual values where available
    for state in h2_state_cap.index:
        df_full.loc[state] = h2_state_cap.loc[state]

    # Prepare for merging - add network identifier
    df_full["network"] = network
    all_h2_state_caps.append(df_full)

# Process the merged data
if all_h2_state_caps:
    # Concatenate all dataframes
    merged_h2_caps = pd.concat(all_h2_state_caps, ignore_index=False)
    merged_h2_caps = merged_h2_caps.reset_index()
    merged_h2_caps.rename(columns={"index": "State"}, inplace=True)

    # Reorder columns to have 'network' and 'State' first
    cols = ["network", "State"] + [
        col for col in merged_h2_caps.columns if col not in ["network", "State"]
    ]
    merged_h2_caps = merged_h2_caps[cols]

    # Melt the DataFrame to long format for pivoting
    df_melted = merged_h2_caps.melt(
        id_vars=["network", "State"],
        value_vars=["Alkaline electrolyzer large", "PEM electrolyzer", "SOEC"],
        var_name="Technology",
        value_name="Value",
    )

    # Create pivot table with MultiIndex columns (network, technology)
    df_pivoted = df_melted.pivot(
        index="State", columns=["network", "Technology"], values="Value"
    )

    # Sort and clean up the result
    df_pivoted = df_pivoted.sort_index()  # Sort states alphabetically
    df_pivoted = df_pivoted.sort_index(
        axis=1, level=[0, 1]
    )  # Sort columns by network, then technology
    df_pivoted = df_pivoted.fillna(0)  # Fill NaN with 0

    save_to_excel_with_formatting(
        df_pivoted,
        "Electrolyzer Capacity By State",
        "Electrolyzer Capacity By State (MW)",
        excel_file_path,
        freeze_pane="B4",
    )
else:
    print("No hydrogen capacity data found across scenarios.")

In [ ]:
# First, collect all unique grid regions across all scenarios
all_grid_regions = set()
for network in networks.keys():
    h2_grid_cap = (
        compute_h2_capacities(networks[network])
        .groupby(["grid_region"])[
            ["Alkaline electrolyzer large", "PEM electrolyzer", "SOEC"]
        ]
        .sum()
    )
    all_grid_regions.update(h2_grid_cap.index)

all_grid_regions = sorted(list(all_grid_regions))  # Sort for consistency

all_h2_grid_caps = []

for idx, network in enumerate(networks.keys()):
    year = network[-4:]
    h2_grid_cap = (
        compute_h2_capacities(networks[network])
        .groupby(["grid_region"])[
            ["Alkaline electrolyzer large", "PEM electrolyzer", "SOEC"]
        ]
        .sum()
    )

    print(
        f"\nInstalled electrolyzer capacity (MW) by Grid Region - {network} (only Grid Regions ≥ 10 kW))\n"
    )

    # Transpose and filter states with capacity >= 0.01 MW (10 kW) for display only
    df_display = h2_grid_cap.sort_values(
        by=list(h2_grid_cap.columns), ascending=False
    ).T
    df_display.columns.name = "Grid Region"
    df_display.index.name = None
    df_display = df_display.loc[:, (df_display >= 0.01).any(axis=0)]

    display(df_display.style.format("{:.1e}"))

    # For merging: Create a full DataFrame with all grid regions, filling missing with 0
    df_full = pd.DataFrame(
        index=all_grid_regions,
        columns=["Alkaline electrolyzer large", "PEM electrolyzer", "SOEC"],
    ).fillna(0)
    # Update with actual values where available
    for grid_region in h2_grid_cap.index:
        df_full.loc[grid_region] = h2_grid_cap.loc[grid_region]

    # Prepare for merging - add network identifier
    df_full["network"] = network
    all_h2_grid_caps.append(df_full)

# Process the merged data
if all_h2_grid_caps:
    # Concatenate all dataframes
    merged_h2_grid_caps = pd.concat(all_h2_grid_caps, ignore_index=False)
    merged_h2_grid_caps = merged_h2_grid_caps.reset_index()
    merged_h2_grid_caps.rename(columns={"index": "Grid Region"}, inplace=True)

    # Reorder columns to have 'network' and 'Grid Region' first
    cols = ["network", "Grid Region"] + [
        col
        for col in merged_h2_grid_caps.columns
        if col not in ["network", "Grid Region"]
    ]
    merged_h2_grid_caps = merged_h2_grid_caps[cols]

    # Melt the DataFrame to long format for pivoting
    df_melted = merged_h2_grid_caps.melt(
        id_vars=["network", "Grid Region"],
        value_vars=["Alkaline electrolyzer large", "PEM electrolyzer", "SOEC"],
        var_name="Technology",
        value_name="Value",
    )

    # Create pivot table with MultiIndex columns (network, technology)
    df_pivoted = df_melted.pivot(
        index="Grid Region", columns=["network", "Technology"], values="Value"
    )

    # Sort and clean up the result
    df_pivoted = df_pivoted.sort_index()  # Sort grid regions alphabetically
    df_pivoted = df_pivoted.sort_index(
        axis=1, level=[0, 1]
    )  # Sort columns by network, then technology
    df_pivoted = df_pivoted.fillna(0)  # Fill NaN with 0

    save_to_excel_with_formatting(
        df_pivoted,
        "Electrolyzer Capacity By Grid",
        "Electrolyzer Capacity By Grid (MW)",
        excel_file_path,
        freeze_pane="B5",
    )

    print(f"\nExported merged hydrogen capacity by grid region to {excel_file_path}")

else:
    print("No hydrogen capacity data found across scenarios.")

#### Production and Consumption of Hydrogen

In [ ]:
combined_h2_summary, per_scenario_h2_tables = compute_h2_balance_tables(
    networks,
    energy_carriers=["H2", "grid H2", "e-kerosene", "NH3"],
    unit="MWh",
    plot=True,
)

# Save combined summary to Excel
save_to_excel_with_formatting(
    combined_h2_summary,
    "H2_Efuels_Balance_Summary",
    "Hydrogen & e-fuels Balance Summary (MWh)",
    excel_file_path,
    freeze_pane="B4",
)

# display a per-scenario summary
for scen, data in per_scenario_h2_tables.items():
    print(f"\nScenario: {scen}\n")
    display(data["summary"].style.format("{:,.0f}"))

# display combined (MultiColumn) summary
print("\nCombined summary (scenarios = top-level columns):\n")
display(combined_h2_summary.style.format("{:,.0f}"))

### 4.3. e-kerosene production capacity
*This sector reports information about installed capacity for Fischer-Tropsch synthesis plants for e-kerosene production.*

In [ ]:
ft_state_table = {}

for name, net in networks.items():
    fig, ax, data = create_ft_capacity_by_state_map(
        net.copy(),
        path_shapes=state_shapes_path,
        network_name=name,
        min_capacity_gw=0.4,
        year_title=False,
    )
    ax.set_aspect("equal", adjustable="datalim")
    showfig()

    # data['grid_region'] = data.bus0.map(n.buses.grid_region)
    ft_state_capacity = data.groupby("state")["p_nom_gw"].sum()
    ft_state_table[name] = ft_state_capacity

In [ ]:
# Representative plant capacity in MW (can be changed easily)
plant_capacity_mw = 400

df = pd.DataFrame(ft_state_table)
df = df.reset_index()
df.rename(columns={df.columns[0]: "State"}, inplace=True)
df = df.set_index("State")

df = df.where(df >= 0.4, 0)
df_plants = np.floor(df * 1000 / plant_capacity_mw).astype(int)

# Build a MultiIndex DataFrame: top level = year, second level = metric
tuples = []
data = {}
for net_col in df.columns:
    tuples.extend([(net_col, "Capacity (GW input H2)"), (net_col, "N. of plants")])
    data[(net_col, "Capacity (GW input H2)")] = df[net_col].astype(float)
    data[(net_col, "N. of plants")] = df_plants[net_col].astype(int)

df_combined = pd.DataFrame(data)


save_to_excel_with_formatting(
    df_combined,
    "FT_Capacity By State",
    "Fisher Tropsch Capacity By State (GW)",
    excel_file_path,
    freeze_pane="B4",
)

print(f"\nExported merged FT capacity by state to {excel_file_path}")


# Style: highlight cells where plants >= 1 (both capacity and plants)
def highlight_df(df):
    styles = pd.DataFrame("", index=df.index, columns=df.columns)
    for col in df.columns:
        if "N. of plants" in col:
            styles[col] = df[col].apply(
                lambda v: "background-color: #fff2b2;"
                if isinstance(v, (int, float)) and v >= 1
                else ""
            )
        if "Capacity" in col:
            year = col.split(" - ")[1]
            plant_col = f"N. of plants - {year}"
            styles[col] = df[plant_col].apply(
                lambda v: "background-color: #fff2b2;" if v >= 1 else ""
            )
    return styles


styled_df = df_combined.style.apply(highlight_df, axis=None)

display(
    Markdown(
        f"\nInstalled FT capacity by State (GW input H2) "
        f"and equivalent number of representative plants ({plant_capacity_mw} MW each)\n"
    )
)
display(styled_df)

In [ ]:
ft_grid_table = {}

for name, net in networks.items():
    fig, ax, data = create_ft_capacity_by_grid_region_map(
        net.copy(),
        path_shapes=grid_region_shapes_path,
        network_name=name,
        min_capacity_gw=0.4,
        year_title=False,
    )
    ax.set_aspect("equal", adjustable="datalim")
    showfig()

    ft_grid_table[name] = data.groupby("grid_region")["total_gw"].sum()

In [ ]:
df = pd.DataFrame(ft_grid_table)
df = df.reset_index()
df.rename(columns={df.columns[0]: "Grid Region"}, inplace=True)
df = df.set_index("Grid Region")

df = df.where(df >= 0.4, 0)
df_plants = np.floor(df * 1000 / plant_capacity_mw).astype(int)

# Build a MultiIndex DataFrame: top level = year, second level = metric
tuples = []
data = {}
for net_col in df.columns:
    tuples.extend([(net_col, "Capacity (GW input H2)"), (net_col, "N. of plants")])
    data[(net_col, "Capacity (GW input H2)")] = df[net_col].astype(float)
    data[(net_col, "N. of plants")] = df_plants[net_col].astype(int)

df_combined = pd.DataFrame(data)

save_to_excel_with_formatting(
    df_combined,
    "FT_Capacity By Grid",
    "Fischer Tropsch Capacity By Grid (GW)",
    excel_file_path,
)

print(f"\nExported merged FT capacity by grid region to {excel_file_path}")

styled_df = df_combined.style.apply(highlight_df, axis=None)

display(
    Markdown(
        f"\nInstalled FT capacity by Grid Region (GW input H2) "
        f"and equivalent number of representative plants ({plant_capacity_mw} MW each)\n"
    )
)
display(styled_df)

---

##  5. Operational analysis

*This section explores electricity and hydrogen generation and dispatch.*

### 5.1. Electricity dispatch
*Electricity dispatch is assessed for each single source across the model years.*

In [ ]:
elec_dispatch = plot_electricity_dispatch(
    networks, tech_colors, nice_names, title_year=False, return_data=True
)

In [ ]:
if elec_dispatch:
    dispatch_tables = elec_dispatch["dispatch"]
    demand_tables = elec_dispatch["demand"]

    # Create multi-level DataFrames
    dispatch_df = pd.concat(dispatch_tables, axis=1)
    demand_df = pd.concat(demand_tables, axis=1)

    # Add 'Total Demand' as a row in dispatch for each network
    combined_tables = {}
    for key in dispatch_tables.keys():
        # Get dispatch data for this network
        dispatch_data = dispatch_tables[key]
        demand_data = demand_tables[key]

        # Add demand as a new row
        combined_data = dispatch_data.copy()
        combined_data.loc[:, "Total Demand"] = demand_data.values

        combined_tables[key] = combined_data

    # Concatenate all networks
    final_combined_df = pd.concat(combined_tables, axis=1)
    save_to_excel_with_formatting(
        final_combined_df,
        "Electricity Dispatch",
        "Electricity Dispatch (GW)",
        excel_file_path,
        freeze_pane="B4",
    )

### 5.2. Electricity generation
*Total electricity generation by source is reported by year, with an indication of the percentage covered by renewable or clean (renewables + nuclear) energy sources per State and at country-wise level.*

In [ ]:
generation_summary = {}

for key, network in networks.items():
    # year = key[-4:]  # Extract the year
    generation_summary[key] = calculate_total_generation_by_carrier(network)

generation_df = pd.DataFrame.from_dict(generation_summary, orient="index").fillna(0)
generation_df = generation_df.sort_index().round(2)

generation_df.columns = [nice_names.get(c, c) for c in generation_df.columns]

preferred_order = [
    "Coal",
    "Oil",
    "Gas OCGT",
    "Gas CCGT",
    "Nuclear",
    "Biomass",
    "Conventional hydro",
    "Run-of-River hydro",
    "Geothermal",
    "Utility-scale solar",
    "Rooftop solar",
    "Onshore wind",
    "Offshore wind",
    "CSP",
]

available_cols = generation_df.columns.tolist()
ordered_cols = [c for c in preferred_order if c in available_cols] + [
    c for c in available_cols if c not in preferred_order
]

generation_df = generation_df[ordered_cols]
generation_df["Total"] = generation_df.sum(axis=1).round(2)

print("\nTotal electricity generation (TWh/year) by technology\n")
display(generation_df.T)

save_to_excel_with_formatting(
    generation_df.T,
    "Electricity_Generation",
    "Total electricity generation (TWh/year) by technology",
    excel_file_path,
)

print(f"\nExported Electricity Generation to {excel_file_path}")

### 5.3. CO2 emissions from electricity generation

In [ ]:
# CO2 intensity [tCO2 per MWh_fuel] for original link carriers
co2_intensity = {
    "coal": 0.3361,
    "oil": 0.2571,
    "OCGT": 0.198,
    "CCGT": 0.198,
    "urban central gas CHP": 0.198,
}

# carriers handled via Links (same list you used for generation)
link_carriers = [
    "coal",
    "oil",
    "OCGT",
    "CCGT",
    "biomass",
    "lignite",
    "urban central solid biomass CHP",
    "urban central gas CHP",
]

# Prepare data structures
emissions_data = defaultdict(dict)  # {(scenario, year): {metric: value}}
scenarios_years = set()  # To track all (scenario, year) combinations

for net_key, net in networks.items():
    # Extract scenario and year from net_key (e.g., "scenario_01_2030" -> scenario="scenario_01", year=2030)
    match = re.search(r"(?:scenario_(\d{2})|Base)_(\d{4})", net_key)
    if match:
        if match.group(1):
            scenario = f"scenario_{match.group(1)}"
        else:
            scenario = "Base"
        year = int(match.group(2))
    else:
        print(
            f"Warning: Skipping network '{net_key}' - does not match expected format (e.g., 'scenario_01_2030' or 'Base_2023')."
        )
        continue

    # --- snapshots and timestep (same logic as your generation function) ---
    snapshots_slice = slice(None)
    snapshots = net.snapshots[snapshots_slice]
    if len(snapshots) < 2:
        raise ValueError(f"Network {net_key} has insufficient snapshots")
    timestep_h = (snapshots[1] - snapshots[0]).total_seconds() / 3600

    # --- identify electric buses (same filter you used) ---
    electric_buses = set(
        net.buses.index[
            ~net.buses.carrier.str.contains(
                "heat|gas|H2|oil|coal", case=False, na=False
            )
        ]
    )

    # accumulator (tCO2)
    emissions_per_carrier = {c: 0.0 for c in co2_intensity.keys()}

    # --- links: take electrical output on bus1 (p1), same as your generation ---
    for carrier in link_carriers:
        if carrier not in co2_intensity:
            continue  # we only emit for the carriers in co2_intensity

        links = net.links[
            (net.links.carrier == carrier) & (net.links.bus1.isin(electric_buses))
        ]
        if links.empty:
            continue

        # electrical power on bus1; keep only positive electrical output
        p1 = net.links_t.p1.loc[snapshots_slice, links.index]
        p1_positive = -p1.clip(upper=0)  # same trick: make export positive

        # ELECTRICAL energy [MWh]
        e_el_mwh = p1_positive.sum().sum() * timestep_h

        # average efficiency to back-calculate fuel input
        # (if per-link efficiencies differ a lot, weight them by energy)
        eta_series = links["efficiency"].replace({None: 1.0})
        # simple energy-weighted mean efficiency (robust):
        # compute fuel per-link, then sum
        # fuel_input = sum( e_el_link / eta_link )
        # To do that, compute per-link electrical energy first:
        e_el_per_link = p1_positive.sum(axis=0) * timestep_h  # MWh per link

        # avoid division by zero / NaN
        valid = eta_series.notna() & (eta_series > 0)
        e_el_per_link = e_el_per_link[valid]
        eta_series = eta_series[valid]

        fuel_input_mwh = (e_el_per_link / eta_series).sum()

        # emissions [tCO2]
        emissions_tco2 = fuel_input_mwh * co2_intensity[carrier]
        emissions_per_carrier[carrier] += emissions_tco2

    # --- aggregate into Oil / Gas / Coal buckets (GtCO2) ---
    oil_val = emissions_per_carrier.get("oil", 0.0) / 1e9
    coal_val = emissions_per_carrier.get("coal", 0.0) / 1e9
    gas_val = (
        emissions_per_carrier.get("OCGT", 0.0)
        + emissions_per_carrier.get("CCGT", 0.0)
        + emissions_per_carrier.get("urban central gas CHP", 0.0)
    ) / 1e9
    total_val = oil_val + gas_val + coal_val

    # Store data
    emissions_data[(scenario, year)] = {
        "Oil": oil_val,
        "Gas": gas_val,
        "Coal": coal_val,
        "Total": total_val,
        "Total (Statistics)": 1.421 if year == 2023 else "-",
    }

    # Track scenarios and years
    scenarios_years.add((scenario, year))

# Build the MultiIndex DataFrame
# Get all unique (scenario, year) combinations, sorted by scenario then year
sorted_scenarios_years = sorted(scenarios_years, key=lambda x: (x[0], x[1]))

# Create columns: (scenario, year)
columns = []
for scenario, year in sorted_scenarios_years:
    columns.append((scenario, year))

# Create the DataFrame with metrics as index
emissions_df = pd.DataFrame(
    index=["Oil", "Gas", "Coal", "Total", "Total (Statistics)"],
    columns=pd.MultiIndex.from_tuples(columns),
)

# Populate the DataFrame
for (scenario, year), data in emissions_data.items():
    for metric, value in data.items():
        formatted_value = (
            f"{value:.3f}"
            if isinstance(value, (int, float)) and not pd.isna(value)
            else value
        )
        emissions_df.at[metric, (scenario, year)] = formatted_value


def custom_format(x):
    if isinstance(x, (int, float)) and not pd.isna(x):
        return f"{x:.2f}"
    else:
        return str(x)  # Convert to string for strings, NaN, etc.


emissions_df = (
    emissions_df.replace("-", np.nan).astype(float, errors="ignore").fillna(0)
)
if 2023 in emissions_df.columns.levels[1]:
    base_2023 = emissions_df.loc["Total", ("Base", 2023)]
    emissions_df.loc["Reduction vs 2023 (%)"] = (
        (emissions_df.loc["Total", :].values - base_2023) / base_2023 * 100
    )
else:
    emissions_df.loc["Reduction vs 2023 (%)"] = np.nan

save_to_excel_with_formatting(
    emissions_df,
    "CO2_Emissions",
    "CO2 Emissions from Electricity Generation (GtCO2)",
    excel_file_path,
    freeze_pane="B4",
)

styled_df = emissions_df.style.format(custom_format, na_rep="N/A")
print(
    "\nCO2 Emissions from Electricity Generation (GtCO2) - Comparison across all scenarios"
)
display(styled_df)

### 5.4. Renewable / Clean electricity generation shares

In [ ]:
eia_generation_data = pd.read_excel(
    "./data/validation_data/annual_generation_state.xls", skiprows=1
)
eia_generation_data_df = preprocess_res_ces_share_eia(eia_generation_data)[
    ["% Actual RES", "% Actual CES"]
]

grid_regions = pd.read_excel("./data/validation_data/grid_regions_to_states.xlsx")
grid_regions["States"] = grid_regions["States"].str.strip().str.upper()

In [ ]:
ces_path = os.path.join(project_root, "data", "res_ces_shares", "clean_targets.csv")
res_path = os.path.join(project_root, "data", "res_ces_shares", "res_targets.csv")

ces = pd.read_csv(ces_path, index_col=0)
res = pd.read_csv(res_path, index_col=0)

ces.columns = ces.columns.astype(str)
res.columns = res.columns.astype(str)

ces.index = ces.index.str.strip().str.upper()
res.index = res.index.str.strip().str.upper()

res_carriers = [
    "solar",
    "onwind",
    "offwind-ac",
    "offwind-dc",
    "ror",
    "hydro",
    "geothermal",
    "solar rooftop",
]
ces_carriers = res_carriers + ["nuclear"]

In [ ]:
# Call the function and store the DataFrame
stored_state_df = display_state_results(
    networks,
    eia_generation_data_df,
    ces,
    res,
    ces_carriers=ces_carriers,
    res_carriers=res_carriers,
)
stored_state_df = (
    stored_state_df.replace("N/A", np.nan).astype(float, errors="ignore").round(2)
)

save_to_excel_with_formatting(
    stored_state_df,
    "RES_CES_Analysis by State",
    "Merged Analysis of State-wise Renewable / Clean Electricity Generation Targets based on State",
    excel_file_path,
    freeze_pane="B5",
)

In [ ]:
combined_df_grid_res_share_table, _ = display_grid_region_results_multiple_scenario(
    networks=networks,
    ces=ces,
    res=res,
    ces_carriers=ces_carriers,
    res_carriers=res_carriers,
)

save_to_excel_with_formatting(
    combined_df_grid_res_share_table,
    "RES_CES_Analysis by Grid",
    "Merged Analysis of Grid-wise Renewable / Clean Electricity Generation Targets based on Grid Region",
    excel_file_path,
    freeze_pane="B5",
)

In [ ]:
combined_df_grid_res_share_table

---

### 5.5. Hydrogen dispatch

*Similar to electricity dispatch, the dispatch of hydrogen through the model years is assessed with detail on the technologies used for its production.*

In [ ]:
h2_carriers = [
    "Alkaline electrolyzer large",
    "Alkaline electrolyzer medium",
    "Alkaline electrolyzer small",
    "PEM electrolyzer",
    "SOEC",
]

In [ ]:
hydrogen_dispatch_df = plot_hydrogen_dispatch(networks, h2_carriers, year_title=False)


save_to_excel_with_formatting(
    hydrogen_dispatch_df,
    "Hydrogen_Dispatch",
    "Hourly Hydrogen Dispatch (tons) by Technology and Scenario",
    excel_file_path,
    freeze_pane="B4",
)

### 5.6. Hourly Matching

In [ ]:
results_country = analyze_additionality_multiple_networks(
    networks=networks,
    regions=[None],  # Only whole country
    additionality=True,
    nice_names_power=nice_names_power,
    nice_names=nice_names,
    tech_power_color=tech_power_color,
    tech_colors=tech_colors,
    show_plots=True,
    skip_years=[2023],  # Skip 2023 if hourly matching not implemented
)

In [ ]:
dfs_for_concat = {}

for network_name, regions_dict in results_country.items():
    # Get the dataframe (first element of tuple) for 'whole_country'
    df, fig = regions_dict["whole_country"]
    dfs_for_concat[network_name] = df

# Create multi-column dataframe with scenario at top level
df_additionality_all = pd.concat(dfs_for_concat, axis=1, keys=dfs_for_concat.keys())
save_to_excel_with_formatting(
    df_additionality_all,
    "Additionality_Analysis_Country",
    "Additionality Analysis - Whole Country",
    excel_file_path,
    freeze_pane="B4",
)

In [ ]:
results_subplots = analyze_additionality_multiple_subplots(
    networks=networks,
    regions=None,  # Auto-detect regions, or pass explicit list
    include_whole_country=True,  # Add whole country as a subplot
    additionality=True,
    nice_names_power=nice_names_power,
    nice_names=nice_names,
    tech_power_color=tech_power_color,
    tech_colors=tech_colors,
    skip_years=[2023],
    show_plots=True,
    ncols=2,  # Number of columns in subplot grid
    subplot_height=4,  # Height of each subplot
    subplot_width=9,  # Width of each subplot
)

In [ ]:
scenario_dfs = {}

for network_name, (dataframes_dict, fig) in results_subplots.items():
    if not dataframes_dict:
        print(f"[WARN] {network_name}: no regional dataframes, skipped")
        continue

    scenario_df = pd.concat(dataframes_dict, axis=1, keys=dataframes_dict.keys())
    scenario_dfs[network_name] = scenario_df

if not scenario_dfs:
    raise RuntimeError("No scenarios produced any dataframes")

df_additionality_regions_all = pd.concat(scenario_dfs, axis=1, keys=scenario_dfs.keys())

save_to_excel_with_formatting(
    df_additionality_regions_all,
    "Additionality_Analysis_Regions",
    "Additionality Analysis - All Regions",
    excel_file_path,
    freeze_pane="B4",
)

### 5.7 Hydrogen VS e-kerosene production

In [ ]:
hydrogen_ekerosene_prod = {}
for name, net in networks.items():
    hydrogen_ekerosene_prod[name] = compare_h2_kerosene_production(
        net, network_name=name
    )

In [ ]:
#  loop through hydrogen_ekerosene_prod and concatenate H2 and kerosene production data
hydrogen_ekerosene = {}

for scenario, data in hydrogen_ekerosene_prod.items():
    h2_prod_df = pd.DataFrame(data["h2_production"]).rename(
        columns={0: "H2 Production (GW)"}
    )
    kerosene_prod_df = pd.DataFrame(data["kerosene_production"]).rename(
        columns={0: "Kerosene Production (GW)"}
    )
    combined_df = pd.concat([h2_prod_df, kerosene_prod_df], axis=1)
    hydrogen_ekerosene[scenario] = combined_df

# merge all scenarios into a single DataFrame with MultiIndex columns
merged_h2_kerosene_df = pd.concat(
    hydrogen_ekerosene, axis=1, keys=hydrogen_ekerosene.keys()
)
save_to_excel_with_formatting(
    merged_h2_kerosene_df,
    "H2_Kerosene_Production",
    "Hourly Hydrogen and Kerosene Production (GW)",
    excel_file_path,
    freeze_pane="B4",
)

In [ ]:
h2_capacity_factors = {}

print("\nCapacity factor for H2 production\n")
for name, net in networks.items():
    # compute CF for this network
    results = compute_capacity_factor_electrolysis({name: net})
    h2_capacity_factors[name] = results

    for y, res in results.items():
        # select regional aggregation
        df = res["regions"]

        # filter regions with negligible capacity
        df_filtered = df[df["Capacity (GW input electricity)"] >= 1e-3].copy()
        if df_filtered.empty:
            continue

        df_filtered.reset_index(drop=True, inplace=True)

        print(f"\nYear: {y}\n")
        display(
            df_filtered.style.hide(axis="index").format(
                {
                    "Capacity (GW input electricity)": "{:.2f}",
                    "Electricity input (MWh)": "{:.0f}",
                    "Hydrogen output (MWh)": "{:.0f}",
                    "Capacity factor (%)": "{:.2f}",
                }
            )
        )

In [ ]:
# combined_h2_capacity_factors = pd.concat(h2_capacity_factors, axis=1, keys=h2_capacity_factors.keys())
# combined_h2_capacity_factors

### 5.8. Deliverability

In [ ]:
electrolysis_carriers = [
    "Alkaline electrolyzer large",
    "Alkaline electrolyzer medium",
    "Alkaline electrolyzer small",
    "PEM electrolyzer",
    "SOEC",
]

temporal_matching_carriers = [
    "csp",
    "solar",
    "onwind",
    "offwind-ac",
    "offwind-dc",
    "ror",
    "hydro",
    "nuclear",
]

In [ ]:
# ============================================================
# Electrolyzers deliverability – robust multi-scenario plotting
# ============================================================

import pandas as pd
import matplotlib.pyplot as plt

electrolyzers_develiverablilty_dfs = {}

# Select networks to plot (skip Base / 2023)
networks_to_plot = [
    (name, n) for name, n in networks.items() if not name.endswith("2023")
]

if len(networks_to_plot) == 0:
    raise RuntimeError("No scenario networks to plot (all networks are Base/2023?)")

# Collect all grid regions appearing in ANY scenario
all_regions = set()

for _, n in networks_to_plot:
    electrolyzers = n.links[n.links.carrier.isin(electrolysis_carriers)]
    n.links["grid_region"] = n.links.bus0.map(n.buses.grid_region)

    elec_input = (
        n.links_t.p0[electrolyzers.index].groupby(n.links.grid_region, axis=1).sum()
        / 1e3
    )

    all_regions.update(elec_input.columns)

regions = sorted(all_regions)

n_rows = len(regions)
n_cols = len(networks_to_plot)

# Create figure grid (DATA-DRIVEN)
fig, ax = plt.subplots(
    nrows=n_rows,
    ncols=n_cols,
    figsize=(6 * n_cols, 4 * n_rows),
    sharey=False,
    squeeze=False,
)

# First pass: global max for y-axis
global_max = 0.0

for network_name, n in networks_to_plot:
    electrolyzers = n.links[n.links.carrier.isin(electrolysis_carriers)]
    n.links["grid_region"] = n.links.bus0.map(n.buses.grid_region)

    elec_input = (
        n.links_t.p0[electrolyzers.index].groupby(n.links.grid_region, axis=1).sum()
        / 1e3
    )

    n.generators["grid_region"] = n.generators.bus.map(n.buses.grid_region)
    res_gens = n.generators[n.generators.carrier.isin(temporal_matching_carriers)]

    res_gen_output = (
        n.generators_t.p[res_gens.index].groupby(n.generators.grid_region, axis=1).sum()
        / 1e3
    )

    if not elec_input.empty:
        global_max = max(global_max, elec_input.max().max())
    if not res_gen_output.empty:
        global_max = max(global_max, res_gen_output.max().max())

# Plotting
for j, (network_name, n) in enumerate(networks_to_plot):
    electrolyzers = n.links[n.links.carrier.isin(electrolysis_carriers)]
    n.links["grid_region"] = n.links.bus0.map(n.buses.grid_region)

    elec_input = (
        n.links_t.p0[electrolyzers.index].groupby(n.links.grid_region, axis=1).sum()
        / 1e3
    )

    avg_elec_input = elec_input.mean()

    n.generators["grid_region"] = n.generators.bus.map(n.buses.grid_region)
    res_gens = n.generators[n.generators.carrier.isin(temporal_matching_carriers)]

    res_gen_output = (
        n.generators_t.p[res_gens.index].groupby(n.generators.grid_region, axis=1).sum()
        / 1e3
    )

    # Store data for Excel
    electrolyzers_develiverablilty_dfs[network_name] = pd.concat(
        [elec_input, res_gen_output], axis=1, keys=["elec_input", "res_gen_output"]
    )

    for i, region in enumerate(regions):
        # If region not present in this network → disable subplot
        if region not in elec_input.columns:
            ax[i, j].axis("off")
            continue

        elec_input[region].plot(
            ax=ax[i, j],
            label=f"Electrolyzers input (avg: {avg_elec_input[region]:.2f} GW)",
        )

        if region in res_gen_output.columns:
            res_gen_output[region].plot(ax=ax[i, j], label="RES output")

        ax[i, j].set_title(f"{network_name}, Region: {region}")
        ax[i, j].set_ylabel("Power (GW)")
        ax[i, j].set_ylim(0, global_max * 1.05)

        # Month ticks
        start = elec_input.index.min().replace(day=1)
        end = elec_input.index.max()
        month_starts = pd.date_range(start=start, end=end, freq="MS")

        ax[i, j].set_xlim(start, end)
        ax[i, j].set_xticks(month_starts)
        ax[i, j].set_xticklabels(month_starts.strftime("%b"))
        ax[i, j].tick_params(axis="x", rotation=0)
        ax[i, j].set_xlabel(None)

        # Legend only on top row
        if i == 0:
            ax[i, j].legend()
        else:
            ax[i, j].legend().remove()

plt.tight_layout()

# ------------------------------------------------------------
# Export to Excel (one sheet, all scenarios/years)
# ------------------------------------------------------------
electrolyzers_develiverablilty_dfs_combined = pd.concat(
    electrolyzers_develiverablilty_dfs, axis=1
)

save_to_excel_with_formatting(
    electrolyzers_develiverablilty_dfs_combined,
    "Electrolyzers Deliverability",
    "Electrolyzers Deliverability (GW)",
    excel_file_path,
    freeze_pane="B4",
)

## 6. Economic Analysis

*This section shows the evolution of levelized costs of electricity and hydrogen throughout the model time horizon.*

### 6.1. Marginal prices of electricity

In [ ]:
region_prices_dfs = {}
for key, net in networks.items():
    region_prices = extract_marginal_price_by_grid_region_weighted(net)
    plot_marginal_prices_by_region_weighted(
        region_prices,
        network=net,
        network_name=key,
        demand_threshold=0.01,
        year_title=False,
    )
    region_prices_dfs[key] = region_prices

In [ ]:
combined_region_prices_dfs = pd.concat(
    region_prices_dfs, axis=1, keys=region_prices_dfs.keys()
)
save_to_excel_with_formatting(
    combined_region_prices_dfs,
    "Marginal_Prices_by_Grid_Region",
    "Marginal Prices by Grid Region (USD/MWh)",
    excel_file_path,
    freeze_pane="B4",
)
print(f"\nExported Marginal Prices by Grid Region to {excel_file_path}")

### 6.2. Levelized Cost of Electricity (LCOE)
*The LCOE is assessed at the level of the different US grid regions.*

In [ ]:
grid_regions_shapes = gpd.read_file(grid_region_shapes_path)

In [ ]:
# Collect LCOE values and cache results
all_weighted_lcoe = []
cached_results = {}
collected_lcoe_tables = {}

for key, net in networks.items():
    lcoe_gdf, table, lcoe_by_bus, lcoe_data, *_ = calculate_lcoe_summary_and_map(
        net, grid_regions_shapes
    )
    cached_results[key] = (net, lcoe_gdf, table, lcoe_by_bus, lcoe_data)

    # Map EMM regions and transmission fees
    # emm_mapping = dict(zip(net.buses.grid_region, net.buses.emm_region))
    table["EMM Region"] = table["grid_region"].map(emm_mapping)

    fee_map = regional_fees.loc[
        regional_fees["Year"] == int(key.split("_")[-1]),
        ["region", "Transmission nom USD/MWh", "Distribution nom USD/MWh"],
    ].set_index("region")[["Transmission nom USD/MWh", "Distribution nom USD/MWh"]]

    table["Transmission (USD/MWh)"] = table["EMM Region"].map(
        fee_map.loc[:, "Transmission nom USD/MWh"]
    )
    table["Distribution (USD/MWh)"] = table["EMM Region"].map(
        fee_map.loc[:, "Distribution nom USD/MWh"]
    )
    table.drop(columns=["EMM Region"], inplace=True)

    table.loc[:, "LCOE_transmission_fees (USD/MWh)"] = (
        table.loc[:, "Weighted Average LCOE (USD/MWh)"]
        + table.loc[:, "Transmission (USD/MWh)"]
    )
    table.loc[:, "LCOE_transmission_distribution_fees (USD/MWh)"] = (
        table.loc[:, "LCOE_transmission_fees (USD/MWh)"]
        + table.loc[:, "Distribution (USD/MWh)"]
    )

    table = table.rename(
        columns={"Transmission (USD/MWh)": "Transmission fees (USD/MWh)"}
    )
    table = table.rename(
        columns={"Distribution (USD/MWh)": "Distribution fees (USD/MWh)"}
    )
    table = table.rename(
        columns={
            "LCOE_transmission_fees (USD/MWh)": "LCOE + Transmission fees (USD/MWh)"
        }
    )
    table = table.rename(
        columns={
            "LCOE_transmission_distribution_fees (USD/MWh)": "LCOE + T&D fees (USD/MWh)"
        }
    )

    collected_lcoe_tables[key] = table

    merged = lcoe_by_bus.merge(lcoe_data[["bus", "energy"]], on="bus", how="left")
    grouped = merged.groupby("grid_region").apply(
        lambda df: (df["weighted_lcoe"] * df["energy"]).sum() / df["energy"].sum()
    )
    all_weighted_lcoe.extend(grouped.dropna().values)

# Collect all unique grid regions across all scenarios
all_grid_regions = set()
for df in collected_lcoe_tables.values():
    if not df.empty:
        all_grid_regions.update(df["grid_region"].unique())
all_grid_regions = sorted(list(all_grid_regions))

columns = [
    "CCGT lcoe (USD/MWh)",
    "biomass lcoe (USD/MWh)",
    "coal lcoe (USD/MWh)",
    "geothermal lcoe (USD/MWh)",
    "hydro lcoe (USD/MWh)",
    "nuclear lcoe (USD/MWh)",
    "onwind lcoe (USD/MWh)",
    "ror lcoe (USD/MWh)",
    "solar lcoe (USD/MWh)",
    "solar rooftop lcoe (USD/MWh)",
    "urban central gas CHP lcoe (USD/MWh)",
    "urban central solid biomass CHP lcoe (USD/MWh)",
    "CCGT dispatch (TWh)",
    "biomass dispatch (TWh)",
    "coal dispatch (TWh)",
    "geothermal dispatch (TWh)",
    "hydro dispatch (TWh)",
    "nuclear dispatch (TWh)",
    "onwind dispatch (TWh)",
    "ror dispatch (TWh)",
    "solar dispatch (TWh)",
    "solar rooftop dispatch (TWh)",
    "urban central gas CHP dispatch (TWh)",
    "urban central solid biomass CHP dispatch (TWh)",
    "Weighted Average LCOE (USD/MWh)",
    "Transmission fees (USD/MWh)",
    "Distribution fees (USD/MWh)",
    "LCOE + Transmission fees (USD/MWh)",
    "LCOE + T&D fees (USD/MWh)",
]

dfs = []
for scenario in networks.keys():
    if scenario in collected_lcoe_tables and not collected_lcoe_tables[scenario].empty:
        df_indexed = collected_lcoe_tables[scenario].set_index("grid_region")
    else:
        df_indexed = pd.DataFrame(index=all_grid_regions, columns=columns).fillna(
            np.nan
        )
    dfs.append(df_indexed)

merged_lcoe_df = pd.concat(dfs, keys=networks.keys(), axis=1)
save_to_excel_with_formatting(
    merged_lcoe_df,
    "Weighted Average LCOE",
    "Levelized Cost of Electricity (LCOE) Summary by Grid Region and Scenario",
    excel_file_path,
    freeze_pane="B4",
)

In [ ]:
# Compute global vmin/vmax
vmin = np.quantile(all_weighted_lcoe, 0.05)
vmax = np.quantile(all_weighted_lcoe, 0.95)

# Plot using cached data
plots_data = {}

for key, (net, lcoe_gdf, table, lcoe_by_bus, lcoe_data) in cached_results.items():
    year_match = re.search(r"\d{4}", key)
    year_str = year_match.group() if year_match else "Year N/A"
    title = f"Weighted average (plant level) LCOE per Grid Region (USD/MWh) - {key}"

    fig, ax = plt.subplots(
        figsize=(12, 10), subplot_kw={"projection": ccrs.PlateCarree()}
    )
    plots_data[key] = table

    plot_lcoe_map_by_grid_region(
        lcoe_by_bus,
        lcoe_data,
        grid_regions_shapes,
        title=title,
        key=key,
        ax=ax,
        vmin=vmin,
        vmax=vmax,
    )

    showfig()

In [ ]:
def uppercase_only_lcoe_word(df):
    new_cols = {
        col: re.sub(r"\blcoe\b", "LCOE", col, flags=re.IGNORECASE) for col in df.columns
    }
    return df.rename(columns=new_cols)


def apply_nice_names_to_columns_custom(df, mapping):
    # Ordina per lunghezza decrescente per sostituire prima le chiavi più specifiche
    sorted_mapping = sorted(mapping.items(), key=lambda item: -len(item[0]))

    placeholder_prefix = "__PLACEHOLDER__"
    placeholder_map = {}

    new_cols = []
    for col in df.columns:
        temp_col = col
        for i, (key, val) in enumerate(sorted_mapping):
            if key in temp_col:
                placeholder = f"{placeholder_prefix}{i}__"
                placeholder_map[placeholder] = val
                temp_col = temp_col.replace(key, placeholder)
        new_cols.append(temp_col)

    final_cols = [
        reduce(lambda c, p: c.replace(p[0], p[1]), placeholder_map.items(), col)
        for col in new_cols
    ]

    df.columns = final_cols
    return df


for net_name, table in plots_data.items():
    # year = net_name[-4:]
    print(f"\nYear: ({net_name})")

    if table.index.name == "grid_region":
        table = table.reset_index()

    table = table.rename(columns={"grid_region": "Grid Region"})
    table = table.rename(
        columns={"Transmission (USD/MWh)": "Transmission fees (USD/MWh)"}
    )
    table = table.rename(
        columns={"Distribution (USD/MWh)": "Distribution fees (USD/MWh)"}
    )
    table = table.rename(
        columns={
            "LCOE_transmission_fees (USD/MWh)": "LCOE + Transmission fees (USD/MWh)"
        }
    )
    table = table.rename(
        columns={
            "LCOE_transmission_distribution_fees (USD/MWh)": "LCOE + T&D fees (USD/MWh)"
        }
    )

    table.index = pd.Index([""] * len(table))

    table = uppercase_only_lcoe_word(table)
    table = apply_nice_names_to_columns_custom(table, nice_names)

    display(table.style.format(precision=2, thousands=","))

In [ ]:
collected_lcoe_tables = {}

for key, net in networks.items():
    _, _, _, lcoe_data, *_ = calculate_lcoe_summary_and_map(net, grid_regions_shapes)

    region_summary = (
        lcoe_data.groupby(["grid_region", "bus", "carrier"])
        .agg(
            dispatch_mwh=("energy", "sum"),
            total_cost=("lcoe", lambda x: (x * lcoe_data.loc[x.index, "energy"]).sum()),
        )
        .reset_index()
    )

    region_summary["lcoe"] = (
        region_summary["total_cost"] / region_summary["dispatch_mwh"]
    )
    region_summary["dispatch"] = region_summary["dispatch_mwh"] / 1e6
    region_summary["bus"] = region_summary["bus"].str.extract(r"(US\d{1} \d{1,2})")

    region_bus_summary = region_summary.pivot(
        index=["grid_region", "bus"], columns="carrier", values=["lcoe", "dispatch"]
    )
    region_bus_summary.columns = [
        f"{carrier} {metric} ({'USD/MWh' if metric == 'lcoe' else 'TWh'})"
        for metric, carrier in region_bus_summary.columns
    ]
    region_bus_summary = region_bus_summary.reset_index()

    dispatch_cols = [
        col for col in region_bus_summary.columns if "dispatch" in col.lower()
    ]
    for col in dispatch_cols:
        region_bus_summary[col] = pd.to_numeric(
            region_bus_summary[col], errors="coerce"
        ).fillna(0.0)

    lcoe_cols = [col for col in region_bus_summary.columns if "lcoe" in col.lower()]

    min_dispatch_threshold = 1  # TWh
    for lcoe_col in lcoe_cols:
        carrier = lcoe_col.split(" ")[0]
        dispatch_col = next(
            (col for col in dispatch_cols if col.startswith(carrier + " ")), None
        )
        if dispatch_col:
            mask = region_bus_summary[dispatch_col] < min_dispatch_threshold
            region_bus_summary.loc[mask, lcoe_col] = np.nan

    table_df = (
        region_bus_summary.groupby(["grid_region"])
        .agg(["min", "max"])
        .drop(columns=["bus"])
    )
    collected_lcoe_tables[key] = table_df

    plot_float_bar_lcoe_dispatch_ranges(
        table_df, key, nice_names, use_scenario_names=True
    )

merged_lcoe_df = pd.concat(collected_lcoe_tables, axis=1)
save_to_excel_with_formatting(
    merged_lcoe_df,
    "LCOE_Ranges_by_Grid_Region",
    "LCOE Ranges by Grid Region and Scenario",
    excel_file_path,
    freeze_pane="B5",
)

In [ ]:
collected_national_lcoe_tables = {}

for key, net in networks.items():
    # Recalculate lcoe_data and region_bus_summary
    _, _, _, lcoe_data, *_ = calculate_lcoe_summary_and_map(net, grid_regions_shapes)

    region_summary = (
        lcoe_data.groupby(["grid_region", "bus", "carrier"])
        .agg(
            dispatch_mwh=("energy", "sum"),
            total_cost=("lcoe", lambda x: (x * lcoe_data.loc[x.index, "energy"]).sum()),
        )
        .reset_index()
    )

    region_summary["lcoe"] = (
        region_summary["total_cost"] / region_summary["dispatch_mwh"]
    )
    region_summary["dispatch"] = region_summary["dispatch_mwh"] / 1e6
    region_summary["bus"] = region_summary["bus"].str.extract(r"(US\d{1} \d{1,2})")

    region_bus_summary = region_summary.pivot(
        index=["grid_region", "bus"], columns="carrier", values=["lcoe", "dispatch"]
    )
    region_bus_summary.columns = [
        f"{carrier} {metric} ({'USD/MWh' if metric == 'lcoe' else 'TWh'})"
        for metric, carrier in region_bus_summary.columns
    ]
    region_bus_summary = region_bus_summary.reset_index()

    # Convert dispatch to numeric and apply threshold filtering to LCOE
    dispatch_cols = [
        col for col in region_bus_summary.columns if "dispatch" in col.lower()
    ]
    lcoe_cols = [col for col in region_bus_summary.columns if "lcoe" in col.lower()]

    for col in dispatch_cols:
        region_bus_summary[col] = pd.to_numeric(
            region_bus_summary[col], errors="coerce"
        ).fillna(0.0)

    min_dispatch_threshold = 1  # TWh
    for lcoe_col in lcoe_cols:
        carrier = lcoe_col.split(" ")[0]
        dispatch_col = next(
            (col for col in dispatch_cols if col.startswith(carrier + " ")), None
        )
        if dispatch_col:
            mask = region_bus_summary[dispatch_col] < min_dispatch_threshold
            region_bus_summary.loc[mask, lcoe_col] = np.nan

    # --- NATIONAL PLOT ONLY ---
    national_data = region_bus_summary.drop(columns=["grid_region", "bus"])
    lcoe_columns = [col for col in national_data.columns if "lcoe" in col.lower()]
    national_data = national_data[lcoe_columns]

    national_min = national_data.min()
    national_max = national_data.max()

    arrays = [
        [col for col in national_min.index] * 2,
        ["min"] * len(national_min) + ["max"] * len(national_max),
    ]
    multi_cols = pd.MultiIndex.from_tuples(zip(*arrays))

    national_values = pd.concat([national_min, national_max])
    national_df = pd.DataFrame(
        [national_values.values], columns=multi_cols, index=["USA"]
    )
    collected_national_lcoe_tables[key] = national_df

    display(
        Markdown(
            "\n**Note:** The LCOE values reported here also include existing nuclear plants, many of which are already amortized and were built under different historical cost conditions. In contrast, Lazard’s estimates reflect newly built projects with current capital costs and construction standards, which explains the higher LCOE values."
        )
    )
    plot_float_bar_lcoe_dispatch_ranges(
        national_df, key, nice_names, use_scenario_names=True
    )

merged_national_lcoe_df = pd.concat(collected_national_lcoe_tables, axis=1)
merged_national_lcoe_df = merged_national_lcoe_df.sort_index(axis=1, level=0)
save_to_excel_with_formatting(
    merged_national_lcoe_df,
    "LCOE_Ranges_National",
    "National LCOE Ranges by Scenario",
    excel_file_path,
    freeze_pane="B5",
)

### 6.3. Marginal prices of hydrogen
*The marginal price of hydrogen computed by the model is assessed at the level of the different US Grid Regions.*

In [ ]:
grid_regions_shapes = gpd.read_file(grid_region_shapes_path)

In [ ]:
region_price = compute_marginal_h2_price_by_grid_region(
    networks=networks,
    h2_carriers=h2_carriers,
    regional_fees=regional_fees,
    emm_mapping=emm_mapping,
    output_threshold=1.0,
    include_baseload=True,
    year_title=False,
)

In [ ]:
plot_marginal_h2_price_maps(
    region_price=region_price,
    grid_regions_shapes=grid_regions_shapes,
)

In [ ]:
tables_by_year = build_marginal_h2_price_table(region_price)
marginal_h2_prices_by_grid_region = pd.concat(
    tables_by_year, axis=1, keys=tables_by_year.keys()
)

save_to_excel_with_formatting(
    marginal_h2_prices_by_grid_region,
    "H2_Marginal_Price_by_Grid_Region",
    "Marginal H2 Prices by Grid Region (USD/kg H2)",
    excel_file_path,
    freeze_pane="B4",
)

In [ ]:
display(marginal_h2_prices_by_grid_region)

### 6.4. Levelized Cost of Hydrogen (LCOH)
*The levelized cost of hydrogen is assessed at the level of the different US Grid Regions.*

In [ ]:
plot_lcoh_maps_by_grid_region_marginal(
    networks,
    grid_regions_shapes,
    h2_carriers,
    regional_fees,
    emm_mapping,
    output_threshold=0,
    year_title=False,
)

In [ ]:
lcoe_gdf, table, lcoe_by_bus, lcoe_data, vmin, vmax = calculate_lcoe_summary_and_map(
    n, grid_regions_shapes
)

grid_region_weighted_lcoe = (
    lcoe_by_bus.merge(lcoe_data[["bus", "energy"]], on="bus", how="left")
    .groupby("grid_region")
    .apply(lambda df: (df["weighted_lcoe"] * df["energy"]).sum() / df["energy"].sum())
)

res_lcoe = calculate_lcoh_by_region(
    networks,
    h2_carriers,
    regional_fees,
    emm_mapping,
    electricity_price="LCOE",
    grid_region_lcoe=grid_region_weighted_lcoe,
    include_baseload=True,  # Include baseload charges
    baseload_charge_path="./data/energy_charge_rate.csv",
    customer_charge_mw=400,  # in MW
    demand_charge_rate=9.0,  # in $/kW/month
    year_title=False,
)

In [ ]:
lcoe_gdf, table, lcoe_by_bus, lcoe_data, vmin, vmax = calculate_lcoe_summary_and_map(
    n, grid_regions_shapes
)

# Using LCOE as electricity cost
plot_lcoh_maps_by_grid_region_lcoe(
    networks=networks,
    shapes=grid_regions_shapes,
    h2_carriers=h2_carriers,
    regional_fees=regional_fees,
    emm_mapping=emm_mapping,
    grid_region_lcoe=grid_region_weighted_lcoe,
    output_threshold=0,
    year_title=False,
)

In [ ]:
# LCOH using marginal cost as electricity price
res_marginal = calculate_lcoh_by_region(
    networks,
    h2_carriers,
    regional_fees,
    emm_mapping,
    include_baseload=True,
    electricity_price="marginal",
    year_title=False,
)

collected_lcoh_marginal_cost = {}

print("\nLCOH using marginal cost as electricity price")
for year, df in res_marginal.items():
    print(f"\nYear {year}:")
    collected_lcoh_marginal_cost[year] = df

    # format only numeric columns
    numeric_cols = df.select_dtypes(include="number").columns
    fmt = {col: "{:.2f}" for col in numeric_cols}

    display(df.style.format(fmt).hide(axis="index"))

# Collect all unique grid regions across all scenarios
all_grid_regions = set()
for df in collected_lcoh_marginal_cost.values():
    if not df.empty:
        all_grid_regions.update(df["Grid Region"].unique())
all_grid_regions = sorted(list(all_grid_regions))

columns = [
    "Electrolysis CAPEX (USD/kg H2)",
    "Electrolysis OPEX (USD/kg H2)",
    "Electricity (USD/kg H2)",
    "LCOH (excl. T&D fees) (USD/kg H2)",
    "Transmission fees (USD/kg H2)",
    "LCOH + Transmission fees (USD/kg H2)",
    "LCOH + Transmission fees + Baseload charges (USD/kg H2)",
    "Distribution fees (USD/kg H2)",
    "LCOH + T&D fees (USD/kg H2)",
    "Baseload charges (USD/kg H2)",
    "LCOH + Transmission + Baseload (USD/kg H2)",
    "Hydrogen Dispatch (tons per region)",
]

dfs = []
for scenario in networks.keys():
    if (
        scenario in collected_lcoh_marginal_cost
        and not collected_lcoh_marginal_cost[scenario].empty
    ):
        df_indexed = collected_lcoh_marginal_cost[scenario].set_index("Grid Region")
    else:
        df_indexed = pd.DataFrame(index=all_grid_regions, columns=columns).fillna(
            np.nan
        )
    dfs.append(df_indexed)

merged_lcoh_marginal_cost = pd.concat(dfs, keys=networks.keys(), axis=1)
save_to_excel_with_formatting(
    merged_lcoh_marginal_cost,
    "LCOH_Marginal_Cost",
    "LCOH using marginal cost as electricity price",
    excel_file_path,
    freeze_pane="B4",
)

In [ ]:
# Compute LCOE per bus and per region
lcoe_gdf, table, lcoe_by_bus, lcoe_data, vmin, vmax = calculate_lcoe_summary_and_map(
    n, regions_onshore
)

# Weighted average LCOE per grid region (USD/MWh el)
grid_region_weighted_lcoe = (
    lcoe_by_bus.merge(lcoe_data[["bus", "energy"]], on="bus", how="left")
    .groupby("grid_region")
    .apply(lambda df: (df["weighted_lcoe"] * df["energy"]).sum() / df["energy"].sum())
)

# LCOH using LCOE as electricity price
res_lcoe = calculate_lcoh_by_region(
    networks,
    h2_carriers,
    regional_fees,
    emm_mapping,
    include_baseload=True,
    electricity_price="LCOE",
    grid_region_lcoe=grid_region_weighted_lcoe,
    year_title=False,
)

collected_lcoh_lcoe = {}

print("\nLCOH using LCOE as electricity price")
for year, df in res_lcoe.items():
    print(f"\nYear {year}:")

    collected_lcoh_lcoe[year] = df

    # format only numeric columns
    numeric_cols = df.select_dtypes(include="number").columns
    fmt = {col: "{:.2f}" for col in numeric_cols}

    display(df.style.format(fmt).hide(axis="index"))

# Collect all unique grid regions across all scenarios
all_grid_regions = set()
for df in collected_lcoh_lcoe.values():
    if not df.empty:
        all_grid_regions.update(df["Grid Region"].unique())
all_grid_regions = sorted(list(all_grid_regions))

columns = [
    "Electrolysis CAPEX (USD/kg H2)",
    "Electrolysis OPEX (USD/kg H2)",
    "Electricity (USD/kg H2)",
    "LCOH (excl. T&D fees) (USD/kg H2)",
    "Transmission fees (USD/kg H2)",
    "LCOH + Transmission fees (USD/kg H2)",
    "LCOH + Transmission fees + Baseload charges (USD/kg H2)",
    "Distribution fees (USD/kg H2)",
    "LCOH + T&D fees (USD/kg H2)",
    "Baseload charges (USD/kg H2)",
    "LCOH + Transmission + Baseload (USD/kg H2)",
    "Hydrogen Dispatch (tons per region)",
]

dfs = []
for scenario in networks.keys():
    if scenario in collected_lcoh_lcoe and not collected_lcoh_lcoe[scenario].empty:
        df_indexed = collected_lcoh_lcoe[scenario].set_index("Grid Region")
    else:
        df_indexed = pd.DataFrame(index=all_grid_regions, columns=columns).fillna(
            np.nan
        )
    dfs.append(df_indexed)

merged_lcoh_lcoe_df = pd.concat(dfs, keys=networks.keys(), axis=1)
save_to_excel_with_formatting(
    merged_lcoh_lcoe_df,
    "LCOH_lcoe_as_Cost",
    "LCOH using LCOE as electricity price",
    excel_file_path,
    freeze_pane="B4",
)

---

## 7. E-Kerosene

*This section is dedicated to the analysis of e-kerosene production and costs.*

### 7.1. Aviation fuel demand

In [ ]:
df = compute_aviation_fuel_demand(
    networks, include_scenario=True, scenario_as_index=True, wide=True
).fillna(0)
save_to_excel_with_formatting(
    df,
    "Aviation_Fuel_Demand",
    "Aviation Fuel Demand by Scenario (TWh)",
    excel_file_path,
    freeze_pane="B4",
)

styled_df = (
    df.style.hide(axis="index")
    .format(
        {
            "Kerosene (TWh)": "{:,.2f}",
            "e-Kerosene (TWh)": "{:,.2f}",
            "Total (TWh)": "{:,.2f}",
            "e-Kerosene Share (%)": "{:.1f}%",
        }
    )
    .set_properties(**{"text-align": "right", "font-size": "13px"})
    .set_table_styles(
        [
            {
                "selector": "th",
                "props": [("text-align", "right"), ("font-size", "12px")],
            },
            {
                "selector": "caption",
                "props": [
                    ("caption-side", "top"),
                    ("font-weight", "bold"),
                    ("font-size", "16px"),
                ],
            },
        ]
    )
)

print("\nAviation fuel demand\n")
display(styled_df)

In [ ]:
passenger_data = pd.read_csv(
    "./data/T100_Domestic_Market_and_Segment_Data_-3591723781169319541.csv"
)
total_passengers = passenger_data.passengers.sum()

In [ ]:
ekerosene_demand_state_dfs = {}
for network_name, n in networks.items():
    # print(f"\nYear: {network_name[-4:]}\n")
    ekerosene_demand_state = compute_aviation_demand_table(n)
    ekerosene_demand_state.loc[:, "n. of Passengers"] = (
        ekerosene_demand_state["Share (%)"] * total_passengers / 100
    )
    ekerosene_demand_state.set_index("State", inplace=True)
    ekerosene_demand_state_dfs[network_name] = ekerosene_demand_state

In [ ]:
combined_ekerosene_demand_state_df = pd.concat(
    ekerosene_demand_state_dfs, axis=1, keys=ekerosene_demand_state_dfs.keys()
)
save_to_excel_with_formatting(
    combined_ekerosene_demand_state_df,
    "eKerosene_Demand_by_State",
    "e-Kerosene Demand by State and Scenario (TWh)",
    excel_file_path,
    freeze_pane="B5",
)

In [ ]:
for network_name, n in networks.items():
    print(f"Scenario: {network_name}\n")
    create_aviation_demand_by_state_map(
        n,
        grid_region_shapes_path,
        network_name=network_name,
        min_demand_twh=1.0,
        year_title=False,
    )
    showfig()

In [ ]:
ekerosene_demand_grid_region_dfs = {}
for network_name, n in networks.items():
    # print(f"\nYear: {network_name[-4:]}\n")
    ekerosene_demand_grid_region = compute_aviation_demand_table(n, level="grid_region")
    ekerosene_demand_grid_region.loc[:, "n. of Passengers"] = (
        ekerosene_demand_grid_region["Share (%)"] * total_passengers / 100
    )
    ekerosene_demand_grid_region.set_index("Grid Region", inplace=True)
    ekerosene_demand_grid_region_dfs[network_name] = ekerosene_demand_grid_region

In [ ]:
combined_ekerosene_demand_grid_RG_df = pd.concat(
    ekerosene_demand_grid_region_dfs,
    axis=1,
    keys=ekerosene_demand_grid_region_dfs.keys(),
)
save_to_excel_with_formatting(
    combined_ekerosene_demand_grid_RG_df,
    "eKerosene_Demand_by_grid_RG",
    "e-Kerosene Demand by Grid Region and Scenario (TWh)",
    excel_file_path,
    freeze_pane="B5",
)

In [ ]:
for network_name, n in networks.items():
    print(f"Scenario: {network_name}\n")
    create_aviation_demand_by_grid_region_map(
        n,
        grid_region_shapes_path,
        network_name=network_name,
        min_demand_twh=1.0,
        year_title=False,
    )
    showfig()

### 7.2. Required inputs for e-kerosene production

In [ ]:
# Run calculation
ft_df = calculate_total_inputs_outputs_ft(
    networks, include_scenario=True, wide=True
).fillna(0)

save_to_excel_with_formatting(
    ft_df,
    "CO2_Balance_eKerosene",
    "Electricity / Hydrogen / CO2 Balance for e-Kerosene Production",
    excel_file_path,
    freeze_pane="B4",
)

print("\nElectricity / hydrogen / CO2 balance for e-kerosene production \n")

# Display without index
display(ft_df.style.format(precision=2))

### 7.3. Capacity factor of e-kerosene production plants

In [ ]:
# Load factor FT plants
cf = compute_ft_capacity_factor(networks, year_title=False)

# Collect all unique grid regions across all scenarios
all_grid_regions = set()
for df in cf.values():
    if not df.empty:
        all_grid_regions.update(df["Grid Region"].unique())
all_grid_regions = sorted(list(all_grid_regions))

# Define the columns for the DataFrame
columns = ["Capacity (MW)", "Output (MWh)", "Load factor (%)"]

# Collect all DataFrames, set 'Grid Region' as index for each, filling with NaN if missing
dfs = []
for scenario in networks.keys():
    if scenario in cf and not cf[scenario].empty:
        df_indexed = cf[scenario].set_index("Grid Region")
    else:
        df_indexed = pd.DataFrame(index=all_grid_regions, columns=columns).fillna(
            np.nan
        )
    dfs.append(df_indexed)

# Concatenate into a single MultiIndex DataFrame with scenario as top-level column
final_df = pd.concat(dfs, keys=networks.keys(), axis=1)
save_to_excel_with_formatting(
    final_df,
    "FT plants Load factor",
    "FT plants Load factor",
    excel_file_path,
    freeze_pane="B4",
)

print("FT plants Load factor - Combined\n")

# Format all numeric columns to 2 decimals
numeric_cols = final_df.select_dtypes(include="number").columns
fmt = {col: "{:.2f}" for col in numeric_cols}

# Display the final DataFrame with formatting
display(final_df.style.format(fmt))

### 7.4. Economic analysis

#### 7.4.1. Marginal prices of CO2

In [ ]:
cc_carriers = {
    "ethanol from starch CC",
    "SMR CC",
    "DRI CC",
    "BF-BOF CC",
    "dry clinker CC",
    "DAC",
}

co2_regional = compute_regional_co2_production_capture_and_ft_price(
    networks=networks,
    ft_carrier="Fischer-Tropsch",
    cc_carriers=cc_carriers,
    year_title=False,
    verbose=False,
)

co2_regional_df = pd.concat(co2_regional, axis=1, keys=co2_regional.keys())
save_to_excel_with_formatting(
    co2_regional_df,
    "CO2_Regional_Capture_and_FT_Price",
    "Regional CO2 Production, Capture, and FT Price",
    excel_file_path,
    freeze_pane="B4",
)

In [ ]:
display(co2_regional_df)

#### 7.4.2. Levelized cost of CO2 (LCOC)

In [ ]:
print("LCOC cmputed using marginal price as electricity price")

lcoc_by_region_marginal = compute_LCOC_by_region(
    networks=networks,
    regional_fees=regional_fees,
    emm_mapping=emm_mapping,
    electricity_price="marginal",
    year_title=False,
    captured_threshold_mt=1e-3,
    verbose=False,
)

In [ ]:
# Collect all unique grid regions across all scenarios
all_grid_regions = set()
for df in lcoc_by_region_marginal.values():
    if not df.empty:
        all_grid_regions.update(df["Grid Region"].unique())
all_grid_regions = sorted(list(all_grid_regions))

columns = [
    "Captured CO2 (Mt)",
    "Sequestered CO2 (Mt)",
    "CAPEX (USD/tCO2)",
    "Electricity rate (MWh el / tCO2)",
    "Electricity price (USD/MWh el)",
    "Electricity cost (USD/tCO2)",
    "LCOC excl. T&D fees (USD/tCO2)",
    "Transmission fee (USD/MWh)",
    "Distribution fee (USD/MWh)",
    "Transmission cost (USD/tCO2)",
    "Distribution cost (USD/tCO2)",
    "LCOC incl. T&D fees (USD/tCO2)",
]

dfs = []
for scenario in networks.keys():
    if (
        scenario in lcoc_by_region_marginal
        and not lcoc_by_region_marginal[scenario].empty
    ):
        df_indexed = lcoc_by_region_marginal[scenario].set_index("Grid Region")
    else:
        df_indexed = pd.DataFrame(index=all_grid_regions, columns=columns).fillna(
            np.nan
        )
    dfs.append(df_indexed)

final_df = pd.concat(dfs, keys=networks.keys(), axis=1)
save_to_excel_with_formatting(
    final_df,
    "LCOC_marginal_price",
    "LCOC computed using marginal price as electricity price",
    excel_file_path,
    freeze_pane="B4",
)

In [ ]:
# LCOE per grid region
lcoe_gdf, table, lcoe_by_bus, lcoe_data, vmin, vmax = calculate_lcoe_summary_and_map(
    n, regions_onshore
)
lcoe_by_region = (
    lcoe_by_bus.merge(lcoe_data[["bus", "energy"]], on="bus", how="left")
    .groupby("grid_region")
    .apply(lambda df: (df["weighted_lcoe"] * df["energy"]).sum() / df["energy"].sum())
)

# LCOH per grid region (USD/kg H2)
lcoh_by_region = calculate_lcoh_by_region(
    networks,
    h2_carriers,
    regional_fees,
    emm_mapping,
    electricity_price="marginal",
    include_baseload=True,
)

In [ ]:
print("LCOC computed using LCOE as electricity price")

lcoc_by_region_LCOE = compute_LCOC_by_region(
    networks=networks,
    regional_fees=regional_fees,
    emm_mapping=emm_mapping,
    electricity_price="lcoe",
    lcoe_by_region=lcoe_by_region,
    year_title=False,
    captured_threshold_mt=1e-3,
    verbose=False,
)

In [ ]:
# Collect all unique grid regions across all scenarios
all_grid_regions = set()
for df in lcoc_by_region_LCOE.values():
    if not df.empty:
        all_grid_regions.update(df["Grid Region"].unique())
all_grid_regions = sorted(list(all_grid_regions))

columns = [
    "Captured CO2 (Mt)",
    "Sequestered CO2 (Mt)",
    "CAPEX (USD/tCO2)",
    "Electricity rate (MWh el / tCO2)",
    "Electricity price (USD/MWh el)",
    "Electricity cost (USD/tCO2)",
    "LCOC excl. T&D fees (USD/tCO2)",
    "Transmission fee (USD/MWh)",
    "Distribution fee (USD/MWh)",
    "Transmission cost (USD/tCO2)",
    "Distribution cost (USD/tCO2)",
    "LCOC incl. T&D fees (USD/tCO2)",
]

dfs = []
for scenario in networks.keys():
    if scenario in lcoc_by_region_LCOE and not lcoc_by_region_LCOE[scenario].empty:
        df_indexed = lcoc_by_region_LCOE[scenario].set_index("Grid Region")
    else:
        df_indexed = pd.DataFrame(index=all_grid_regions, columns=columns).fillna(
            np.nan
        )
    dfs.append(df_indexed)

final_df = pd.concat(dfs, keys=networks.keys(), axis=1)
save_to_excel_with_formatting(
    final_df,
    "LCOC_lcoe",
    "LCOC computed using LCOE as electricity price",
    excel_file_path,
    freeze_pane="B4",
)

In [ ]:
final_df

#### 7.4.3. Levelized cost of e-kerosene

In [ ]:
# LCO e-kerosene using marginal prices

print(
    "LCO e-kerosene computed using marginal prices for electricity, hydrogen, and CO2"
)

LCO_ekerosene_by_region = compute_LCO_ekerosene_by_region(
    networks=networks,
    fx_2020=1.14,
    fx_recent=1.08,
    regional_fees=regional_fees,
    emm_mapping=emm_mapping,
    unit="gal",
    year_title=True,
    p_nom_threshold=1,  # MW
    electricity_price="marginal",
    hydrogen_price="marginal",
    co2_price="lcoc",
    lcoh_by_region=lcoh_by_region,
    lcoc_by_region=lcoc_by_region_marginal,
)

In [ ]:
# Collect all unique grid regions across all scenarios
all_grid_regions = set()
for df in LCO_ekerosene_by_region.values():
    if not df.empty:
        all_grid_regions.update(df["Grid Region"].unique())
all_grid_regions = sorted(list(all_grid_regions))

columns = [
    "Production (TWh)",
    "Electricity rate (MWh el / MWh e-ker)",
    "H2 rate (MWh H2 / MWh e-ker)",
    "CO2 rate (tCO2 / MWh e-ker)",
    "Electricity price (USD/MWh el)",
    "Hydrogen price (USD/MWh H2)",
    "CO2 price (USD/tCO2)",
    "Electricity cost (USD/gal e-ker)",
    "Hydrogen cost (USD/gal e-ker)",
    "CO2 cost (USD/gal e-ker)",
    "CAPEX (USD/gal e-ker)",
    "VOM (USD/gal e-ker)",
    "LCO e-kerosene (excl. T&D fees) (USD/gal e-ker)",
    "Transmission fees (USD/gal e-ker)",
    "Distribution fees (USD/gal e-ker)",
    "LCO e-kerosene (incl. T&D fees) (USD/gal e-ker)",
]

dfs = []
for scenario in networks.keys():
    if (
        scenario in LCO_ekerosene_by_region
        and not LCO_ekerosene_by_region[scenario].empty
    ):
        df_indexed = LCO_ekerosene_by_region[scenario].set_index("Grid Region")
    else:
        df_indexed = pd.DataFrame(index=all_grid_regions, columns=columns).fillna(
            np.nan
        )
    dfs.append(df_indexed)

final_df = pd.concat(dfs, keys=networks.keys(), axis=1)
save_to_excel_with_formatting(
    final_df,
    "LCO_ekerosene_by_region",
    "LCO e-kerosene computed using marginal prices for electricity, hydrogen, and CO2",
    excel_file_path,
    freeze_pane="B4",
)

In [ ]:
print("LCO e-kerosene computed using LCOE, LCOH and LCOC as input prices")

LCO_ekerosene_by_region = compute_LCO_ekerosene_by_region(
    networks=networks,
    fx_2020=1.14,
    fx_recent=1.08,
    regional_fees=regional_fees,
    emm_mapping=emm_mapping,
    unit="gal",
    year_title=False,
    p_nom_threshold=1e-3,
    electricity_price="lcoe",
    hydrogen_price="lcoh",
    co2_price="lcoc",
    lcoe_by_region=lcoe_by_region,
    lcoh_by_region=lcoh_by_region,
    lcoc_by_region=lcoc_by_region_marginal,
    verbose=False,
)

In [ ]:
# Collect all unique grid regions across all scenarios
all_grid_regions = set()
for df in LCO_ekerosene_by_region.values():
    if not df.empty:
        all_grid_regions.update(df["Grid Region"].unique())
all_grid_regions = sorted(list(all_grid_regions))

columns = [
    "Production (TWh)",
    "Electricity rate (MWh el / MWh e-ker)",
    "H2 rate (MWh H2 / MWh e-ker)",
    "CO2 rate (tCO2 / MWh e-ker)",
    "Electricity price (USD/MWh el)",
    "Hydrogen price (USD/MWh H2)",
    "CO2 price (USD/tCO2)",
    "Electricity cost (USD/gal e-ker)",
    "Hydrogen cost (USD/gal e-ker)",
    "CO2 cost (USD/gal e-ker)",
    "CAPEX (USD/gal e-ker)",
    "VOM (USD/gal e-ker)",
    "LCO e-kerosene (excl. T&D fees) (USD/gal e-ker)",
    "Transmission fees (USD/gal e-ker)",
    "Distribution fees (USD/gal e-ker)",
    "LCO e-kerosene (incl. T&D fees) (USD/gal e-ker)",
]

dfs = []
for scenario in networks.keys():
    if (
        scenario in LCO_ekerosene_by_region
        and not LCO_ekerosene_by_region[scenario].empty
    ):
        df_indexed = LCO_ekerosene_by_region[scenario].set_index("Grid Region")
    else:
        df_indexed = pd.DataFrame(index=all_grid_regions, columns=columns).fillna(
            np.nan
        )
    dfs.append(df_indexed)

final_df = pd.concat(dfs, keys=networks.keys(), axis=1)
save_to_excel_with_formatting(
    final_df,
    "LCO_ekerosene_by_region",
    "LCO e-kerosene computed using LCOE, LCOH (LCOH + Transmission fees + Baseload charges computed using marginal price of electricity) and LCOC (computed using marginal price of electricity) as input prices",
    excel_file_path,
    freeze_pane="B4",
)

### Levelized Cost of e-Kerosene

The levelized cost of e-kerosene is calculated as:

$$
\text{LCO}_{\text{e-kerosene}} \,[\$/\text{gal}] =
\frac{\sum_t \big(P^{\text{elec}}_t \cdot E^{\text{in}}_t\big)}{\sum_t E^{\text{out}}_t}
+ \frac{\sum_t \big(P^{\text{H2}}_t \cdot H^{\text{in}}_t\big)}{\sum_t E^{\text{out}}_t}
+ \frac{\sum_t \big(P^{\text{CO2}}_t \cdot C^{\text{in}}_t\big)}{\sum_t E^{\text{out}}_t}
+ \frac{CAPEX \cdot P_{\text{nom}}}{\sum_t E^{\text{out}}_t}
+ VOM
$$

---

## Definitions

- $P^{\text{elec}}_t$ = electricity price at time *t* [USD/MWh]  
- $E^{\text{in}}_t$ = electricity input at time *t* [MWh]  

- $P^{\text{H2}}_t$ = hydrogen price at time *t* [USD/MWh]  
- $H^{\text{in}}_t$ = hydrogen input at time *t* [MWh]  

- $P^{\text{CO2}}_t$ = CO₂ price at time *t* [USD/tCO₂]  
- $C^{\text{in}}_t$ = CO₂ input at time *t* [tCO₂]  

- $E^{\text{out}}_t$ = e-kerosene output at time *t* [MWh]  

- $CAPEX$ = annualized capital cost [USD/MW·a]  
- $P_{\text{nom}}$ = installed Fischer–Tropsch plant capacity [MW]  

- $VOM$ = variable O&M cost [USD/MWh]  

- $\eta_{\text{gal}}$ = conversion factor from MWh to gallons  

$$
\eta_{\text{gal}} =
\frac{34 \,\text{MJ/L} \times 3.78541 \,\text{L/gal}}{3600 \,\text{MJ/MWh}}
\;\approx\; 0.0357 \,\text{MWh/gal}
$$


---

## 8. Industrial sector analysis

### 8.1 Industrial production
This section focuses on the production from the different industrial sectors related to e-kerosene production (point-source CO2).

In [ ]:
carriers_of_interest = ["NH3", "ethanol", "DRI", "steel BF-BOF", "steel EAF", "cement"]

In [ ]:
# Specific conversion per carrier
conversion_factors = {
    "cement": (1 / 1000, "kt/year"),
    "DRI": (1 / 1000, "kt/year"),
    "steel BF-BOF": (1 / 1000, "kt/year"),
    "steel EAF": (1 / 1000, "kt/year"),
    "ethanol": (1 / 1000 / 5.1666, "kt/year"),  # 1 MWh = 5.1666 t
    "NH3": (1 / 80.2 * 3600 / 1e6, "Mgallon/year"),  # 1 MWh = 80.2 gallon
}

summary_all = {}
units = {}

for name, network in networks.items():
    filtered_loads = network.loads[
        network.loads["carrier"].isin(carriers_of_interest)
    ].copy()

    filtered_loads["state"] = filtered_loads["bus"].map(network.buses["state"])

    summary = (
        filtered_loads.groupby("carrier")["p_set"]
        .sum()
        .mul(8760)  # convert from MW to MWh/year
        .reindex(carriers_of_interest)
        .fillna(0)
    )

    converted = []
    unit_column = []
    for carrier in carriers_of_interest:
        value = summary.get(carrier, 0)
        factor, unit = conversion_factors.get(carrier, (1, "MWh/year"))
        converted.append(round(value * factor, 2))
        unit_column.append(unit)

    summary_converted = pd.Series(converted, index=carriers_of_interest)
    # year = name[-4:]  # Extract year from network name
    summary_all[name] = summary_converted

    if "Unit" not in units:
        units["Unit"] = unit_column

summary_df = pd.DataFrame(summary_all)
summary_df["Unit"] = units["Unit"]

pd.set_option("display.float_format", "{:,.1f}".format)

print("\nTotal production by industrial product\n")
display(summary_df)

In [ ]:
save_to_excel_with_formatting(
    summary_df,
    "Industrial_Production_Summary",
    "Total Production by Industrial Product",
    excel_file_path,
    freeze_pane="B3",
)

### 8.2 Industrial emissions
This section focuses on the analysis of the CO2 commodity, with particular reference to the industrial sector and the use for e-kerosene production.

In [ ]:
emissions_tables = {}

for key, net in networks.items():
    # year = key[-4:]
    print(f"\nYear: {key}\n")

    df = compute_emissions_from_links(net)

    # Filter rows with emissions != 0 only
    df_nonzero = df[
        (df["CO2 to Atmosphere (Mt CO2/year)"] != 0)
        | (df["CO2 Captured (Mt CO2/year)"] != 0)
        | (df["CO2 Sequestered (Mt CO2/year)"] != 0)
        | (df["Net CO2 Emissions (Mt CO2/year)"] != 0)
    ]

    df_nonzero = df_nonzero.set_index("Process")
    emissions_tables[key] = df_nonzero

    display(df_nonzero.style.format(precision=2).hide(axis="index"))


merged_emissions_df = pd.concat(
    emissions_tables.values(), axis=1, keys=emissions_tables.keys()
)
merged_emissions_df = merged_emissions_df.sort_index(axis=1, level=0)
merged_emissions_df = merged_emissions_df.fillna(0)

save_to_excel_with_formatting(
    merged_emissions_df,
    "Emissions_Summary",
    "CO2 Emissions Summary by Process and Scenario (Mt CO2/year)",
    excel_file_path,
    freeze_pane="B4",
)

In [ ]:
carrier_groups = {
    "BF-BOF": ["BF-BOF", "BF-BOF CC"],
    "DRI": ["DRI", "DRI CC"],
    "ethanol": ["ethanol from starch", "ethanol from starch CC"],
    "SMR": ["SMR", "SMR CC"],
    "dry clinker": ["dry clinker", "dry clinker CC"],
    "DAC": ["DAC"],
}

In [ ]:
# Collect all emissions tables from each scenario
all_dfs = []

for name, net in networks.items():
    df = compute_emissions_by_state(net, carrier_groups)
    # Note: No need to add "scenario" column here since we'll use keys in concat
    all_dfs.append(df)

# Modify each DataFrame to have MultiIndex with state and group
all_dfs_modified = []
for df in all_dfs:
    df_modified = df.reset_index().set_index(["State", "group"])
    all_dfs_modified.append(df_modified)

# Merge the tables into a single MultiIndex DataFrame with scenario at the top-level
merged_emissions_df = pd.concat(all_dfs_modified, axis=1, keys=list(networks.keys()))

# Display the merged DataFrame
print("\nMerged Emissions by State Across All Scenarios\n")
display(
    merged_emissions_df.style.format(precision=2).set_table_styles(
        [
            {
                "selector": "th",
                "props": [("font-weight", "bold"), ("text-align", "center")],
            },
            {"selector": "td", "props": [("text-align", "center")]},
        ]
    )
)

# Optional: Save to Excel with formatting
save_to_excel_with_formatting(
    merged_emissions_df,
    "Merged_Emissions_by_State",
    "Emissions by State Across All Scenarios (Mt CO2/year)",
    excel_file_path,
    freeze_pane="C4",
)

print(f"\nExported merged emissions by state to {excel_file_path}")

# Keep the original plotting code for visualization
global_max = 0
for df in all_dfs:
    max_val = df["Net CO2 Emissions (Mt CO2/year)"].abs().max()
    if max_val > global_max:
        global_max = max_val

for i, (name, df) in enumerate(zip(networks.keys(), all_dfs)):
    scenario = name
    year = scenario[-4:]
    plot_emissions_maps_by_group(
        df, state_shapes_path, scenario, vmin=-global_max, vmax=global_max
    )